#@hidden_cell

SELECT
ORDITEM.ROW_ID AS ORDITEM_ROW_ID,
ACC.OU_TYPE_CD,
ACCI.OU_TYPE_CD AS OU_TYPE_CD_INT,
LE.NAME AS LEGAL_ENTITY,
ACC.INTEGRATION_ID AS CLIENT_ID,
REPLACE(ACC.NAME, ',', '') AS ACC_NAME,
ACC.X_SEGMENT AS ACC_SEGMENT,
IsNull(AST.SERIAL_NUM, '') AS SERIAL_NUM,
IsNull(AST.STATUS_CD, '')  AS ASSET_STATUS,
CASE WHEN AST.REGISTERED_DT IS NOT NULL
THEN CONVERT (VARCHAR (10), AST.REGISTERED_DT, 103)
ELSE
IsNull(CONVERT (VARCHAR (10), ITEMAST.REGISTERED_DT, 103),'')
END AS REG_DATE,
CASE WHEN PROD.PRICE_TYPE_CD = 'One-Time' AND PROD.UOM_CD = 'Month' THEN 1 ELSE ORDITEM.QTY_REQ END AS QTY,
(ORDITEM.NET_PRI * ORDITEM.QTY_REQ) AS ITEM_NET_PRI,
CASE WHEN PROD.PRICE_TYPE_CD = 'Recurring' THEN
      CASE WHEN PARPROD.PR_PROD_LN_ID != PROD.PR_PROD_LN_ID THEN 'Recurring Cover Family Upgrade Bundle' ELSE
            CASE WHEN PROMO.NAME IS NOT NULL THEN
                  CASE WHEN PROMOCOMP.UPG_COUNT > 0 THEN 'Recurring Cover Upgrade Bundle' ELSE 'Recurring Cover Bundle' END
            ELSE 'Recurring Cover' END
      END
ELSE
      CASE WHEN PROD.PRICE_TYPE_CD = 'One-Time' AND PROD.UOM_CD = 'Month' AND ORDITEM.QTY_REQ = 1     THEN 'Cover Bonus Month'
      ELSE
      CASE WHEN PROD.CRT_AGREEMENT_FLG = 'Y' THEN
                  CASE WHEN PARPROD.PR_PROD_LN_ID != PROD.PR_PROD_LN_ID THEN 'Cover Family Upgrade Bundle' ELSE
                        CASE WHEN PROMO.NAME IS NOT NULL THEN
                              CASE WHEN PROMOCOMP.UPG_COUNT > 0 THEN 'Cover Upgrade Bundle' ELSE 'Cover Bundle' END
                        ELSE 'Cover' END
                  END
            WHEN PROD.TYPE = 'Upgrade' AND PARPROD.PR_PROD_LN_ID != PROD.PR_PROD_LN_ID THEN
                        CASE WHEN PROMO.NAME IS NOT NULL THEN 'Family Upgrade Bundle' ELSE 'Family Upgrade' END
            WHEN PROD.TYPE = 'Upgrade' AND PARPROD.PR_PROD_LN_ID = PROD.PR_PROD_LN_ID THEN
                       CASE WHEN PROMO.NAME IS NOT NULL THEN 'Upgrade Bundle' ELSE 'Upgrade' END
            ELSE PROD.TYPE
      END
      END
END AS TRANS_TYPE,
REPLACE(PROD.NAME, '-', '') AS PROD_NAME,
PROD.ROW_ID AS PROD_ROW_ID,
REPLACE(PRODLN.NAME, ',', '') AS PROD_LN_NAME,
PARPROD.NAME AS PARENT_PROD_NAME,
PARPROD.ROW_ID AS PARPROD_ROW_ID,
PARPROD_LN.NAME AS PARENT_PROD_PROD_LINE,
CONVERT (VARCHAR (10), DATEADD(hh, 10, ORD.ORDER_DT), 103) AS TRANS_DATE,
ORD.ORDER_NUM AS REF_NUMBER,
REPLACE(IsNull(PROMO.NAME,''), '-', '') AS BUNDLE_NAME,
IsNull(ORDITEM.ADJ_UNIT_PRI,0) * IsNull(ORDITEM.QTY_REQ,0) AS LIST_PRICE,
CASE WHEN ORD.CREATED_BY = '0-1' THEN 'Web Order'
      ELSE ORDTYPE.NAME
END AS ORDER_TYPE,
CASE WHEN AST.REGISTERED_DT IS NOT NULL
THEN DATEPART(year, AST.REGISTERED_DT)
ELSE
IsNull(DATEPART(year, ITEMAST.REGISTERED_DT),'')
END AS REG_YEAR,
IsNull(ORDITEM.PRI_ADJ_AMT,0) * ORDITEM.QTY_REQ AS POLICY_DISC,
IsNull((SELECT TOP 1 CASE WHEN ORDTYPE.NAME = 'Web Order' AND PAY.PAY_TYPE_CD = 'Credit Card' THEN 'Credit Card - Web' ELSE PAY.PAY_TYPE_CD END AS PAY_CHANNEL
      from S_SRC_PAYMENT PAY with(NOLOCK)
      where ORD.ROW_ID = PAY.ORDER_ID
            AND PAY.PAY_STAT_CD IN ('Authorized','Posted')
            AND PAY.PAY_TYPE_CD != 'Cash'
      order by PAY_AMT DESC), CASE WHEN PROD.PRICE_TYPE_CD = 'Recurring' THEN PAYPRFL.PAY_METH_CD ELSE 'On Account' END) AS PAY_CHANNEL,
IsNull(INV.INVC_NUM,'') AS INV_NUMBER,
IsNull(CONVERT (VARCHAR (10), DATEADD(hh, 10, INV.INVC_DT), 103),'') AS INV_DATE
,   /* get the previous end date*/
    CASE WHEN ITEM.ROW_ID IS NULL THEN '2000-1-1' ELSE
      (select max(DATEADD(hour, 10, a1.EFF_END_DT)) end_dt
      from S_AGREE_ITEM item1 with (NOLOCK)
      INNER JOIN S_DOC_AGREE a1 on a1.ROW_ID=item1.DOC_AGREE_ID
      where item1.ASSET_ID = ITEM.ASSET_ID
      and item1.DOC_AGREE_ID <> AGR.ROW_ID
      and (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
      and a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
      and a1.AGREE_CD != 'Introductory Support' AND a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded') END AS PREV_END_DT
      ,/* Get previous end date ignoring 0 agreement except for bundled 365 product */
      CASE WHEN ITEM.ROW_ID IS NULL
          THEN '2000-1-1'
            ELSE (
                SELECT MAX(DATEADD(hour,10,a1.EFF_END_DT))
                  FROM
                      S_AGREE_ITEM item1 with (NOLOCK)
                        INNER JOIN S_DOC_AGREE a1 ON a1.ROW_ID = item1.DOC_AGREE_ID
                        INNER JOIN S_PROD_INT PROD ON (item1.PROD_INT_ID = PROD.ROW_ID)
                  WHERE
                      item1.ASSET_ID = ITEM.ASSET_ID
                        AND item1.DOC_AGREE_ID <> AGR.ROW_ID
                        AND (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
                        AND a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
                        AND a1.AGREE_CD != 'Introductory Support' AND a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded'
                        AND ((item1.NET_PRI * item1.QTY_REQ) > 0 OR LOWER(PROD.NAME) LIKE '%365 support%')
            )
      END AS PREV_END_DT_V2
      ,/* get the previous agreement row id*/
      CASE WHEN ITEM.ROW_ID IS NULL THEN '2000-1-1' ELSE
      (select top 1 a1.ROW_ID
      from S_AGREE_ITEM item1 with (NOLOCK)
      INNER JOIN S_DOC_AGREE a1 on a1.ROW_ID=item1.DOC_AGREE_ID
      where item1.ASSET_ID = ITEM.ASSET_ID
      and item1.DOC_AGREE_ID <> AGR.ROW_ID
      and (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
      and a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
      and a1.AGREE_CD != 'Introductory Support' AND a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded'
      order by a1.EFF_END_DT desc,a1.STAT_CD desc) END AS PREV_AGREE_ROW_ID
      ,/* Get the previous agreement row id ignoring 0 agreement except for bundle 365 product */
      CASE WHEN ITEM.ROW_ID IS NULL
          THEN '2000-1-1'
            ELSE (
                SELECT TOP 1 a1.ROW_ID
                  FROM
                      S_AGREE_ITEM item1 with (NOLOCK)
                        INNER JOIN S_DOC_AGREE a1 ON (a1.ROW_ID = item1.DOC_AGREE_ID)
                        INNER JOIN S_PROD_INT PROD ON (item1.PROD_INT_ID = PROD.ROW_ID)
                  WHERE
                      item1.ASSET_ID = ITEM.ASSET_ID
                        AND item1.DOC_AGREE_ID <> AGR.ROW_ID
                        AND (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
                        AND a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
                        AND a1.AGREE_CD != 'Introductory Support' AND a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded'
                        AND ((item1.NET_PRI * item1.QTY_REQ) > 0 OR LOWER(PROD.NAME) LIKE '%365 support%')
                  ORDER BY
                      a1.EFF_END_DT DESC, a1.STAT_CD DESC
            )
      END AS PREV_AGREE_ROW_ID_V2

	 , /* Hard code the previous agreement item id for non-Essentials because:
	    - we do not want to match on product id (product in order might be different with agreement product)
		- there can be multiple line items in non-essentials agreement. Essentials should be unlikely.
		- Previous product group for AR is not requested by CommOps atm
		*/
	 NULL AS PREV_AGREE_ITEM_ID

,CASE WHEN ITEM.ROW_ID IS NULL THEN NULL ELSE
      CASE WHEN INIT_AGR.EFF_START_DT IS NOT NULL THEN DATEADD(hh, 10, INIT_AGR.EFF_START_DT) ELSE DATEADD(hh, 10, AGR.EFF_START_DT) END
END AS AGR_START_DT
, /* get the previous one time end date*/
(select max(DATEADD(hour, 10, a1.EFF_END_DT)) end_dt
      from S_AGREE_ITEM item1 with (NOLOCK)
      INNER JOIN S_DOC_AGREE a1 on a1.ROW_ID=item1.DOC_AGREE_ID
      inner join S_PROD_INT PROD ON item1.PROD_INT_ID = PROD.ROW_ID
      where item1.ASSET_ID = ITEM.ASSET_ID
      and item1.DOC_AGREE_ID <> AGR.ROW_ID
      and a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
      and (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
      and a1.AGREE_CD != 'Introductory Support'
      and PROD.PRICE_TYPE_CD = 'One-Time'
      and a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded' ) PREV_ANNUAL_END_DT

      , /* count the number of parallel active agreements*/
(select COUNT(a1.ROW_ID)
      from S_AGREE_ITEM item1 with (NOLOCK)
      INNER JOIN S_DOC_AGREE a1 on a1.ROW_ID=item1.DOC_AGREE_ID
      inner join S_PROD_INT PROD ON item1.PROD_INT_ID = PROD.ROW_ID
      where item1.ASSET_ID = ITEM.ASSET_ID
      and item1.DOC_AGREE_ID <> AGR.ROW_ID
      and a1.EFF_START_DT <= AGR.EFF_START_DT /*agreement has started before or equal to the current agreement*/
      and (a1.EFF_END_DT >= AGR.EFF_START_DT OR a1.EFF_END_DT IS NULL) /*agreement has ended after to the current agreement*/
      and a1.AGREE_CD != 'Introductory Support'
      and item1.NET_PRI > 0 /*Paid agreement*/
      AND a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded') COUNT_OTHER_ACTIVE_PAID_AGREE
,/* get the previous recurring end date*/
(select max(DATEADD(hour, 10, a1.EFF_END_DT)) end_dt
      from S_AGREE_ITEM item1 with (NOLOCK)
      INNER JOIN S_DOC_AGREE a1 on a1.ROW_ID=item1.DOC_AGREE_ID
      inner join S_PROD_INT PROD ON item1.PROD_INT_ID = PROD.ROW_ID
      where item1.ASSET_ID = ITEM.ASSET_ID
      and item1.DOC_AGREE_ID <> AGR.ROW_ID
      and a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
      and (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
      and a1.AGREE_CD != 'Introductory Support'
      and PROD.PRICE_TYPE_CD = 'Recurring' AND a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded') PREV_RECURRING_END_DT
,DIV.NAME AS TEAM
,CAMP.NAME AS CAMPAIGN
,REF.INTEGRATION_ID AS REF_CLIENT_ID
,REF.X_DIVISION AS REF_DIVISION
,REF.X_SEGMENT AS REF_SEGMENT
,REF.NAME AS REF_NAME
,CASE WHEN (ACC.X_DIVISION IN ('Accountants Division') AND EVANG.LOGIN IS NULL)  THEN ACCOUNT_EVANG.LOGIN ELSE EVANG.LOGIN END AS EVANGELIST
,QUOTE.X_COMM_OPTION AS REF_COMM_OPTION
,PROD.UOM_CD AS UOM
,PROD.PART_NUM AS PART_NUM
,REG.SERIAL_NUM AS REG_SERIAL
,ACCOUNT_ACCOUNT_MANAGER_CON.FST_NAME + ' ' +  ACCOUNT_ACCOUNT_MANAGER_CON.LAST_NAME AS ACCOUNT_ACCOUNT_MANAGER
,REF_ACCOUNT_MANAGER_CON.FST_NAME + ' ' +  REF_ACCOUNT_MANAGER_CON.LAST_NAME AS REF_ACCOUNT_MANAGER
,PROD.X_IGNORE_CURR_AGREE_FLG AS IGNORE_CURR_AGREE_FLG
,ITEM.X_USED_BY_ASSET_ID AS AGREE_ITEM_CDF_ROW_ID
,RENEWED_ITEM.ROW_ID AS RENEWED_ITEM_ROW_ID
, /* get the previous end date matched on CDF Id*/
(select max(DATEADD(hour, 10, a1.EFF_END_DT)) end_dt
      from S_AGREE_ITEM item1 with (NOLOCK)
      INNER JOIN S_DOC_AGREE a1 on a1.ROW_ID=item1.DOC_AGREE_ID
      inner join S_PROD_INT PROD ON item1.PROD_INT_ID = PROD.ROW_ID
      where item1.ASSET_ID = ITEM.ASSET_ID
      and item1.DOC_AGREE_ID <> AGR.ROW_ID
      and a1.EFF_END_DT <= DATEADD(day, 5, AGR.EFF_START_DT)
      and (a1.EFF_END_DT <= DATEADD(day, 5, INIT_AGR.EFF_START_DT) OR INIT_AGR.EFF_START_DT IS NULL)
      and a1.AGREE_CD != 'Introductory Support'
      and  a1.STAT_CD != 'Pending' AND a1.STAT_CD != 'Upgraded'
      and item1.X_USED_BY_ASSET_ID = ITEM.X_USED_BY_ASSET_ID) PREV_END_DT_CDF_MATCHED
      ,CASE WHEN PRIMARY_ITEM.DOC_AGREE_ID = AGR.ROW_ID THEN 'Y' ELSE 'N' END AS PRIMARY_AGREE_FLAG
      ,QUOTE.CREATED_BY AS QUOTE_CREATED_BY_ID
      ,QUOTE.QUOTE_NUM AS QUOTE_NUMBER
      ,QUOTE_CREATOR.FST_NAME + ' ' + QUOTE_CREATOR.LAST_NAME AS QUOTE_CREATOR
      ,QUOTE_SALES_REP.FST_NAME + ' ' + QUOTE_SALES_REP.LAST_NAME  AS QUOTE_SALES_REP
      ,POS.NAME AS QUOTE_SALES_REP_POSITION
      ,TEAM.NAME AS SALES_REP_TEAM
      ,ACC.NAME AS CLIENT_NAME
      ,ACC.CREATED AS ACCOUNT_CREATED_DATE,
       (SELECT TOP 1
   QUOTE_CREATOR_TEAM.NAME AS CONTACT_TEAM
   FROM S_POSTN QUOTE_CREATOR_POSTN
   INNER JOIN S_CONTACT QUOTE_CREATOR ON  QUOTE_CREATOR.ROW_ID =  QUOTE_CREATOR_POSTN.PR_EMP_ID
   INNER JOIN S_ORG_EXT QUOTE_CREATOR_TEAM ON QUOTE_CREATOR_POSTN.OU_ID  =  QUOTE_CREATOR_TEAM.ROW_ID
   WHERE QUOTE_CREATOR.ROW_ID = QUOTE.CREATED_BY
   GROUP BY  QUOTE_CREATOR_POSTN.CREATED, QUOTE_CREATOR_TEAM.NAME
   ORDER BY QUOTE_CREATOR_POSTN.CREATED DESC ) AS QUOTE_CREATOR_TEAM
   --,CASE WHEN AGR.STAT_CD IN ('Active', 'Arrears', 'Suspended') THEN 'N' ELSE CASE WHEN (AGR.STAT_CD IS NULL OR PROD.PRICE_TYPE_CD = 'Recurring') THEN 'N/A' ELSE 'Y' END END AS SALE_AGREE_CHURN_FLAG
   ,AGR.AGREE_NUM AS SALE_AGREE_NUM
   ,AGR.EFF_END_DT AS SALE_AGREE_END_DATE
   ,isNull(PROMO.PART_NUM,'') AS BUNDLE_PARTCODE,
   ITEM_OWNER_ACC.INTEGRATION_ID AS CLIENT_ACCOUNT_CLIENT_ID,
      ITEM_OWNER_ACC.NAME AS CLIENT_ACCOUNT_NAME,
      ITEM_OWNER_ACC.X_DIVISION AS CLIENT_ACCOUNT_DIVISION,
      ITEM_OWNER_ACC.X_SEGMENT AS CLIENT_ACCOUNT_SEGMENT,
      ORDER_BILL_ACC.INTEGRATION_ID AS BILL_ACCOUNT_CLIENT_ID,
      ORDER_BILL_ACC.ROW_ID AS BILL_ACCOUNT_ROW_ID,
      ORDER_BILL_ACC.NAME AS BILL_ACCOUNT_NAME,
      ORDER_BILL_ACC.X_DIVISION AS BILL_ACCOUNT_DIVISION,
      ORDER_BILL_ACC.X_SEGMENT AS BILL_ACCOUNT_SEGMENT,
      CASE WHEN ORD.X_REF_ACCT_ID IS NOT NULL THEN 'Y' ELSE 'N' END AS REF_ACCOUNT_ON_ORDER_FLAG,
      CASE WHEN ( SELECT TOP 1 S_ORDER_ITEM.ORDER_ID
      FROM S_ORDER_ITEM
      WHERE S_ORDER_ITEM.X_REL_ASSET_ID IS NOT NULL AND S_ORDER_ITEM.ORDER_ID = ORDITEM.ORDER_ID)  IS NOT NULL THEN 'Y' ELSE 'N' END AS HAS_RELATED_ASSET_FLG,
      ORDER_SALES_REP.FST_NAME + ' ' + ORDER_SALES_REP.LAST_NAME AS ORDER_SALES_REP,
      ORDER_TEAM.NAME AS ORDER_TEAM,
      CASE WHEN AD_PARTNER_SUB_ORDER_CONFIRMED.TARGET_OU_ID IS NULL THEN 'N' ELSE 'Y' END AS AD_PARTNER_SUB_ORDER_CONFIRMED_FLAG,
      AGR.AGREE_NUM AS ORDER_AGREE_NUM,
      AGR.EFF_START_DT AS  ORDER_AGREE_START_DATE,
      AGR.EFF_END_DT AS ORDER_AGREE_END_DATE,
      AGR.STAT_CD AS ORDER_AGREE_STATUS
      ,CONVERT(money, ORDITEM.ADJ_UNIT_PRI) AS START_PRICE
      ,CONVERT(money,(ORDITEM.ADJ_UNIT_PRI - ORDITEM.NET_PRI)) AS DISCOUNT_AMOUNT_CALC
      ,ITEM.ROW_ID AS ORDER_AGRITEM_ID
      ,ACC.ROW_ID AS CLIENT_ROW_ID
      ,REF_EVANG.LOGIN AS REF_REF_EVANGELIST
      ,AGREE_EVANG.LOGIN AS AGREE_EVANGELIST
      ,ORDREF.INTEGRATION_ID AS ORDREF_CLIENT_ID
      ,ORDREF.NAME AS ORDREF_NAME
      ,ORDREF.X_DIVISION AS ORDREF_DIVISION
      ,ORDREF.X_SEGMENT AS ORDREF_SEGMENT
      ,AGREEREF.INTEGRATION_ID AS AGREEREF_CLIENT_ID
      ,AGREEREF.NAME AS AGREEREF_NAME
      ,AGREEREF.X_DIVISION AS AGREEREF_DIVISION
      ,AGREEREF.X_SEGMENT AS AGREEREF_SEGMENT
      ,QUOTE_EVANG_CON.FST_NAME + ' ' + QUOTE_EVANG_CON.LAST_NAME AS QUOTE_EVANG_CON_NAME
      ,EVANG.LOGIN AS QUOTE_EVANG_LOGIN
      ,QUOTE_EVANG_CON_POSTN.NAME AS QUOTE_EVANG_CON_POSTN
      ,QUOTE_EVANG_TEAM.NAME AS QUOTE_EVANG_TEAM
    ,REFACCOUNT_EVANG_CON.FST_NAME + ' ' + REFACCOUNT_EVANG_CON.LAST_NAME AS     REFACCOUNT_EVANG_CON_NAME
      ,REF_EVANG.LOGIN AS REFACCOUNT_EVANG_LOGIN
      ,REFACCOUNT_EVANG_CON_POSTN.NAME AS REFACCOUNT_EVANG_CON_POSTN
      ,REFACCOUNT_EVANG_TEAM.NAME AS REFACCOUNT_EVANG_TEAM
      ,AGREE_EVANG_CON.FST_NAME + ' ' + AGREE_EVANG_CON.LAST_NAME AS AGREE_EVANG_CON_NAME
      , AGREE_EVANG.LOGIN AS AGREE_EVANG_LOGIN
      ,AGREE_EVANG_CON_POS.NAME AS AGREE_EVANG_CON_POS
      ,TEAM_AGREE_EVANG.NAME AS TEAM_AGREE_EVANG_NAME
      ,ACC_EVANG.LOGIN AS ACC_EVANG_LOGIN
      ,REF_CONTACT.FST_NAME + ' ' + REF_CONTACT.LAST_NAME AS REFERRED_BY_CONTACT
	  ,CASE WHEN BUREAU_ASSET.ROW_ID IS NULL THEN 'N' ELSE 'Y' END AS BUREAU_FLG
	  ,AST.ROW_ID AS ASSET_ID
	  ,AST.X_PR_AGREE_ITEM_ID
	  ,AGREE_ITEM.AGREE_ITEM_ID
	  ,CASE WHEN AGREE_ITEM.AGREE_ITEM_ID IS NULL THEN NULL
		ELSE CASE WHEN (AST.X_PR_AGREE_ITEM_ID = AGREE_ITEM.AGREE_ITEM_ID) THEN 'Y' ELSE 'N' END END AS PRIMARY_AGREE_ITEM_FLG
from S_ORDER_ITEM ORDITEM
        left outer join S_ASSET ITEMAST ON ORDITEM.ASSET_ID = ITEMAST.ROW_ID
            left outer join S_INVOICE_ITEM CHILD ON CHILD.ORDER_ITEM_ID = ORDITEM.ROW_ID
            inner join S_ORDER ORD ON ORDITEM.ORDER_ID = ORD.ROW_ID
            inner join S_ORDER_TYPE ORDTYPE ON ORD.ORDER_TYPE_ID = ORDTYPE.ROW_ID
            left outer join S_ORDER_ITEM PARITEM ON ORDITEM.PAR_ORDER_ITEM_ID = PARITEM.ROW_ID
            inner join S_PROD_INT PROD ON ORDITEM.PROD_ID = PROD.ROW_ID AND PROD.TYPE IS NOT NULL
            inner join S_PROD_LN PRODLN ON PROD.PR_PROD_LN_ID = PRODLN.ROW_ID
            left outer join S_PROD_INT PARPROD ON PARITEM.PROD_ID = PARPROD.ROW_ID
            left outer join S_PROD_LN PARPROD_LN ON PARPROD.PR_PROD_LN_ID = PARPROD_LN.ROW_ID
            left outer join S_INVOICE INV ON CHILD.INVOICE_ID = INV.ROW_ID
            left outer join S_PROD_INT PROMO ON ORDITEM.PROMOTION_ID = PROMO.ROW_ID
            left outer join (SELECT PROMOTION_ID AS PROMO_ID, count(*) AS UPG_COUNT
                                    from S_PROM_ITEM with(NOLOCK)
                                    inner join S_PROD_INT ON S_PROM_ITEM.PROD_ID = S_PROD_INT.ROW_ID
                                    where S_PROD_INT.TYPE = 'Upgrade'
                                    group by PROMOTION_ID
                                    )PROMOCOMP ON PROMO.ROW_ID = PROMOCOMP.PROMO_ID
            inner join S_ORG_EXT ACC ON ORD.BILL_ACCNT_ID = ACC.ROW_ID
			inner join S_ORG_EXT ACCI ON ORD.ACCNT_ID = ACCI.ROW_ID
            left outer join S_ASSET AST ON PARITEM.ASSET_ID = AST.ROW_ID
            inner join S_ORG_EXT LE ON ORD.BU_ID = LE.ROW_ID
            /* MATCH TO ORDER AGREEMENT - USE RANK TO GET 1 AGREEMENT ITEM ONLY */
            left outer join ( SELECT AGR.ORDER_ID AS AGR_ORDER_ID, ITEM.ROW_ID AS ITEM_ID, ITEM.ASSET_ID, ITEM.PROD_INT_ID AS ITEM_PROD_ID
                                   ,RANK() OVER(PARTITION BY AGR.ORDER_ID,ITEM.ASSET_ID,ITEM.PROD_INT_ID ORDER BY AGR.CREATED DESC) RANKS
                                    ,RANK() OVER(PARTITION BY ITEM.DOC_AGREE_ID,ITEM.PROD_INT_ID,ITEM.ASSET_ID ORDER BY ITEM.ROW_ID) RANKS2
                                    from S_AGREE_ITEM ITEM with(NOLOCK)
                                    inner join S_DOC_AGREE AGR ON ITEM.DOC_AGREE_ID = AGR.ROW_ID
                                    ) AGRITEM
                                    ON ORD.ROW_ID = AGRITEM.AGR_ORDER_ID AND
                                    ORDITEM.PROD_ID = AGRITEM.ITEM_PROD_ID AND
                                    AST.ROW_ID = AGRITEM.ASSET_ID AND
                                    AGRITEM.RANKS = 1 AND AGRITEM.RANKS2 = 1
          left outer join S_AGREE_ITEM ITEM ON ( AGRITEM.ITEM_ID = ITEM.ROW_ID)
            left outer join S_DOC_AGREE AGR ON ITEM.DOC_AGREE_ID = AGR.ROW_ID
            /* MATCH TO INITIAL AGREEMENT - USE RANK TO GET 1 AGREEMENT ITEM ONLY */
            left outer join (
                                      SELECT DISTINCT
                                      INIT_AGREE.EFF_START_DT,
                                      INIT_AGREE.ORDER_ID,
                                      INIT_AGREE.ROW_ID AS AGREE_ROW_ID,
                                      INIT_AGREE.AGREE_NUM,
                                      S_ASSET.ROW_ID AS ASSET_ROW_ID,
                                      RANK() OVER(PARTITION BY INIT_AGREE.ORDER_ID, S_ASSET.ROW_ID ORDER BY INIT_AGREE.EFF_START_DT ASC) RANKS
                                      FROM S_DOC_AGREE INIT_AGREE
                                      INNER JOIN S_AGREE_ITEM ON S_AGREE_ITEM.DOC_AGREE_ID = INIT_AGREE.ROW_ID
                                      INNER JOIN S_ASSET ON S_ASSET.ROW_ID = S_AGREE_ITEM.ASSET_ID
                                      INNER JOIN S_PROD_INT ON S_AGREE_ITEM.PROD_INT_ID = S_PROD_INT.ROW_ID
                                      WHERE DATEDIFF(day, INIT_AGREE.EFF_START_DT, INIT_AGREE.EFF_END_DT) < 183
                                      AND INIT_AGREE.AGREE_CD != 'Introductory Support'
                                      AND INIT_AGREE.ORDER_ID IS NOT NULL
                                      AND S_PROD_INT.PRICE_TYPE_CD ='One-Time'
                                    ) INIT_AGR
                                      ON ORD.ROW_ID = INIT_AGR.ORDER_ID
                                      AND INIT_AGR.AGREE_ROW_ID != AGR.ROW_ID
                                      AND ITEM.ASSET_ID = INIT_AGR.ASSET_ROW_ID
                                      AND INIT_AGR.RANKS = 1

            left outer join S_DOC_QUOTE QUOTE ON ORD.QUOTE_ID = QUOTE.ROW_ID
            left outer join S_POSTN ACCOUNT_ACCOUNT_MANAGER_POS ON ACCOUNT_ACCOUNT_MANAGER_POS.ROW_ID =  ACC.PR_POSTN_ID
            left outer join S_CONTACT ACCOUNT_ACCOUNT_MANAGER_CON ON ACCOUNT_ACCOUNT_MANAGER_CON.ROW_ID = ACCOUNT_ACCOUNT_MANAGER_POS.PR_EMP_ID
            left outer join S_POSTN POS ON QUOTE.SALES_REP_POSTN_ID = POS.ROW_ID
            left outer join S_ORG_EXT DIV ON POS.OU_ID = DIV.ROW_ID
            left outer join S_SRC CAMP ON ORD.PROMO_ID = CAMP.ROW_ID
            left outer join S_PTY_PAY_PRFL PAYPRFL ON ORD.X_PAYMENT_PROFILE_ID = PAYPRFL.ROW_ID
            left outer join S_AGREE_ITEM RENEWED_ITEM ON RENEWED_ITEM.X_RENEWED_BY_ITEM_ID = ITEM.ROW_ID /*PRIOR RENEWED ITEM*/
            left outer join S_ORG_EXT REF ON QUOTE.X_REF_ACCT_ID = REF.ROW_ID
            left outer join S_USER EVANG ON QUOTE.X_PROD_EVNGLIST_ID = EVANG.ROW_ID
            left outer join REGGIE_COVER_SERIALS REG ON AST.SERIAL_NUM = REG.SERIAL_NUM
            left outer join S_POSTN REF_ACCOUNT_MANAGER_POS ON REF_ACCOUNT_MANAGER_POS.ROW_ID =  REF.PR_POSTN_ID
            left outer join S_CONTACT REF_ACCOUNT_MANAGER_CON ON REF_ACCOUNT_MANAGER_CON.ROW_ID = REF_ACCOUNT_MANAGER_POS.PR_EMP_ID
            left outer join S_AGREE_ITEM PRIMARY_ITEM on PRIMARY_ITEM.ROW_ID = AST.X_PR_AGREE_ITEM_ID
            left outer join S_USER ACCOUNT_EVANG ON ACCOUNT_EVANG.ROW_ID = ACC.X_PROD_EVNGLIST_ID
            left outer join S_CONTACT QUOTE_SALES_REP ON QUOTE_SALES_REP.ROW_ID = POS.PR_EMP_ID
            left outer join S_CONTACT QUOTE_CREATOR ON QUOTE_CREATOR.ROW_ID = QUOTE.CREATED_BY
            left outer join S_ORG_EXT TEAM ON TEAM.PAR_ROW_ID =  POS.OU_ID
            LEFT OUTER JOIN S_ORG_EXT ORDER_BILL_ACC ON ORD.BILL_ACCNT_ID = ORDER_BILL_ACC.ROW_ID
        LEFT OUTER JOIN S_ORG_EXT ITEM_OWNER_ACC ON ORDITEM.OWNER_ACCOUNT_ID = ITEM_OWNER_ACC.ROW_ID
        left outer join S_POSTN ORDER_POS ON ORD.PR_POSTN_ID  = ORDER_POS.ROW_ID
        left outer join S_CONTACT ORDER_SALES_REP ON ORDER_SALES_REP.ROW_ID = ORDER_POS.PR_EMP_ID
        left outer join S_ORG_EXT ORDER_TEAM ON ORDER_POS.OU_ID = ORDER_TEAM.ROW_ID
        LEFT OUTER JOIN (SELECT  A.TARGET_OU_ID, MAX(A.CREATED) AS CREATED FROM S_EVT_ACT A
                                    WHERE A.X_INTERACTION_CD = 'AD Partner Team' AND A.X_INTERACTION2_CD = 'Sub Order Confirm Received'
                                    GROUP BY A.TARGET_OU_ID) AD_PARTNER_SUB_ORDER_CONFIRMED ON AD_PARTNER_SUB_ORDER_CONFIRMED.TARGET_OU_ID = ORDER_BILL_ACC.ROW_ID
            /*QUOTE REFERRED BY CONTACT; OFF QUOTE DIRECTLY */
            left outer join S_CONTACT REF_CONTACT ON REF_CONTACT.ROW_ID = QUOTE.X_REF_CON_ID
            /*EVANGELIST CONTACT, OFF QUOTE DIRECTLY, RATHER THAN ACCOUNT */
            left outer join S_CONTACT QUOTE_EVANG_CON ON QUOTE.X_PROD_EVNGLIST_ID = QUOTE_EVANG_CON.ROW_ID
            left outer join S_POSTN QUOTE_EVANG_CON_POSTN ON QUOTE_EVANG_CON.PR_HELD_POSTN_ID = QUOTE_EVANG_CON_POSTN.ROW_ID
            LEFT OUTER JOIN S_PARTY QUOTE_EVANG_TEAM ON QUOTE_EVANG_CON_POSTN.OU_ID = QUOTE_EVANG_TEAM.PARTY_UID
            /*ORDER REF ACCOUNT; AND EVANGELIST */
            left outer join S_ORG_EXT ORDREF ON ORD.X_REF_ACCT_ID = ORDREF.ROW_ID
            left outer join S_USER REF_EVANG ON ORDREF.X_PROD_EVNGLIST_ID = REF_EVANG.ROW_ID
            left outer join S_CONTACT REFACCOUNT_EVANG_CON ON ORDREF.X_PROD_EVNGLIST_ID = REFACCOUNT_EVANG_CON.ROW_ID
            left outer join S_POSTN REFACCOUNT_EVANG_CON_POSTN ON REFACCOUNT_EVANG_CON_POSTN.ROW_ID = REFACCOUNT_EVANG_CON.PR_HELD_POSTN_ID
            LEFT OUTER JOIN S_PARTY REFACCOUNT_EVANG_TEAM ON REFACCOUNT_EVANG_CON_POSTN.OU_ID = REFACCOUNT_EVANG_TEAM.PARTY_UID
            /*AGREEMENT REF ACCOUNT ; AND EVANGELIST */
            left outer join S_ORG_EXT AGREEREF ON AGR.X_REF_ACCT_ID = AGREEREF.ROW_ID
            left outer join S_USER AGREE_EVANG ON AGREEREF.X_PROD_EVNGLIST_ID = AGREE_EVANG.ROW_ID
            left outer join S_CONTACT AGREE_EVANG_CON ON AGREEREF.X_PROD_EVNGLIST_ID = AGREE_EVANG_CON.ROW_ID
            left outer join S_POSTN AGREE_EVANG_CON_POS ON AGREE_EVANG_CON.PR_HELD_POSTN_ID = AGREE_EVANG_CON_POS.ROW_ID
            LEFT OUTER JOIN S_PARTY TEAM_AGREE_EVANG ON AGREE_EVANG_CON_POS.OU_ID = TEAM_AGREE_EVANG.PARTY_UID
            /*ACCOUNT EVANGELIST */
            LEFT OUTER JOIN S_USER ACC_EVANG ON ACC.X_PROD_EVNGLIST_ID  = ACC_EVANG.ROW_ID
			LEFT OUTER JOIN S_ASSET_REL BUREAU_ASSET ON AST.ROW_ID = BUREAU_ASSET.ASSET_ID AND BUREAU_ASSET.RELATION_TYPE_CD = 'Relates To'
			LEFT OUTER JOIN (SELECT S_DOC_AGREE.ORDER_ID AS AGREE_ORDER_ID, S_AGREE_ITEM.X_ORDER_ITEM_ID, S_AGREE_ITEM.ROW_ID AS AGREE_ITEM_ID
			FROM S_AGREE_ITEM
			INNER JOIN S_DOC_AGREE ON S_AGREE_ITEM.DOC_AGREE_ID = S_DOC_AGREE.ROW_ID ) AGREE_ITEM ON ORDITEM.ROW_ID = AGREE_ITEM.X_ORDER_ITEM_ID AND ORD.ROW_ID = AGREE_ITEM.AGREE_ORDER_ID

where  ORD.ORDER_DT > @month_to_date AND ORD.ORDER_DT  < @month_to_date2
            AND ORDTYPE.NAME IN ('Sales Order', 'Web Order')
            AND ORDITEM.NET_PRI > 0
            AND ORD.STATUS_CD != 'Pending' AND ORD.STATUS_CD != 'Cancelled'
            AND LE.ROW_ID IN ('1-14J-0','1-14J-1')
            AND (PROD.PRICE_TYPE_CD IN ('Recurring',  'One-Time') OR PROD.X_GL_POSTING_GRP = 'MAINTAR')

                                                /*EXCLUDE Referred By Accounts whose account type is 'Internal Customer'  */
                                                AND (ISNULL(REF.OU_TYPE_CD,'NA') <> 'Internal Customer' AND  ISNULL(ORDREF.OU_TYPE_CD,'NA') <> 'Internal Customer'
                                                      AND ISNULL(AGREEREF.OU_TYPE_CD,'NA') <> 'Internal Customer')

            AND PRODLN.NAME IN ('AccountEdge'
                                          ,'Accounting'
                                          ,'AccountingPlus'
                                          ,'Additional Bookkeeper'
                                          ,'Approved Bookkeeper'
                                          ,'AssetManager Pro'
                                          ,'Bookkeeper'
                                          ,'Business Basics'
                                          ,'Certified Consultants'
                                          ,'Developer Program'
                                          ,'Educators'
                                          ,'FirstEdge'
                                          ,'JustInvoices'
                                          ,'Powerpay'
                                          ,'Premier'
                                          ,'Premier Enterprise'
                                          ,'Publishers'
                                          ,'RetailBasics'
                                          ,'RetailHospitality'
                                          ,'RetailManager'
                                          ,'RetailManager Enterprise'
                                          ,'RetailManager Professional'
                                          ,'RetailReady'
                                        ,'AccountEdge NE'
                                        ,'PACs'
                                        ,'Payroll Software'
                                          ,'Cashbook'
                                          ,'Training & Consulting Services'
                                          ,'Training & Consulting Services, Other'
                                          ,'Training - All Classroom Training'
										  ,'Training & Consulting Services - Other'
                                          ,'MYOB Kounta'
                                          ,'Mobile Payments'
																																																		,'Cashbook Online'
																																																        ,'ACE Payroll'
										,'IMS Payroll'
                                          )

                                                                AND (PROD.PART_NUM NOT LIKE '%-POR-%' OR PROD.PART_NUM LIKE '%-POR-INV-%')

## Sales Monitor, this is the first part which extracts the data from OLTP using the store proc.
This is from Line 464 till 898 of the GET_SALES_MONITOR_QUERY_v3

In [1]:
#To extract sales montior data

import numpy as np
import pyodbc
import pandas as pd


#Connection String
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=my_server;DATABASE=my_db;UID=my_user;PWD=my_pwd')
cursor = cnxn.cursor()

## Set Extraction date range

In [2]:
query_from_dt = ("SELECT dateadd(hh,-10,CAST(DATEPART(year, DATEADD(month, -1, GETDATE())) AS VARCHAR) + '-' + CAST(DATEPART(month, DATEADD(month, -1, GETDATE())) AS VARCHAR) + '-1') as FROM_DT")


df_from_dt = pd.read_sql(query_from_dt, cnxn)
from_dt = df_from_dt.iloc[0]['FROM_DT']

str_from_dt = "{:%Y-%m-%d %H:%M:%S}".format(from_dt)
print ("EXTRACT ORDERS FROM DATE '%s'" %str_from_dt)

EXTRACT ORDERS FROM DATE '2018-11-30 14:00:00'


In [3]:
query_to_dt = ("SELECT dateadd(hh,-10,GETDATE()) as TO_DT")


df_to_dt = pd.read_sql(query_to_dt, cnxn)
to_dt = df_to_dt.iloc[0]['TO_DT']

str_to_dt = "{:%Y-%m-%d %H:%M:%S}".format(to_dt)
print ("EXTRACT ORDERS to DATE '%s'" %str_to_dt)

EXTRACT ORDERS to DATE '2019-01-04 04:32:22'


## Order Date filter

In [4]:
order_dt_filter = "ORD.ORDER_DT >'%s' AND ORD.ORDER_DT <'%s'" %(str_from_dt, str_to_dt)
print (order_dt_filter)

ORD.ORDER_DT >'2018-11-30 14:00:00' AND ORD.ORDER_DT <'2019-01-04 04:32:22'


## Order Status filter

In [5]:
order_status_filter = "ORD.STATUS_CD != 'Pending' AND ORD.STATUS_CD != 'Cancelled'"
print (order_status_filter)

ORD.STATUS_CD != 'Pending' AND ORD.STATUS_CD != 'Cancelled'


## Order Item filter

In [6]:
order_item_filter = "ORDITEM.NET_PRI > 0"
print (order_item_filter)

ORDITEM.NET_PRI > 0


## Order Type filter

In [7]:
order_type_filter = "ORDTYPE.NAME IN ('Sales Order' , 'Web order')"
print (order_type_filter)

ORDTYPE.NAME IN ('Sales Order' , 'Web order')


## Product Price type filter
1. EXCLUDE Referred By Accounts whose account type is 'Internal Customer'

In [8]:
product_price_type_filter = "PROD.PRICE_TYPE_CD IN ('Recurring',  'One-Time')"
print (product_price_type_filter)

PROD.PRICE_TYPE_CD IN ('Recurring',  'One-Time')


## Product X_GL_POSTING_GRP filter

In [9]:
prod_x_gl_posting_grp_filter = "PROD.X_GL_POSTING_GRP = 'MAINTAR')"
print (prod_x_gl_posting_grp_filter)

PROD.X_GL_POSTING_GRP = 'MAINTAR')


## Referred by Accounts filter

In [10]:
ref_by_accnt_filter = "(ISNULL(REF.OU_TYPE_CD,'NA') <> 'Internal Customer'"
print (ref_by_accnt_filter)

(ISNULL(REF.OU_TYPE_CD,'NA') <> 'Internal Customer'


## Order referred by Account filter

In [11]:
ord_ref_by_accnt_filter = "ISNULL(ORDREF.OU_TYPE_CD,'NA') <> 'Internal Customer'"
print (ord_ref_by_accnt_filter)

ISNULL(ORDREF.OU_TYPE_CD,'NA') <> 'Internal Customer'


## Agree referred by Account filter

In [12]:
arg_ref_by_accnt_filter = "ISNULL(ARGEEREF.OU_TYPE_CD,'NA') <> 'Internal Customer'"
print (arg_ref_by_accnt_filter)

ISNULL(ARGEEREF.OU_TYPE_CD,'NA') <> 'Internal Customer'


## Order Item Product Line filter

In [13]:
orditem_prod_ln_filter = "PRODLN.NAME IN ('AccountEdge','Accounting','AccountingPlus','Additional Bookkeeper','Approved Bookkeeper','AssetManager Pro','Bookkeeper','Business Basics','Certified Consultants','Developer Program','Educators','FirstEdge','JustInvoices','Powerpay','Premier','Premier Enterprise','Publishers','RetailBasics','RetailHospitality','RetailManager','RetailManager Enterprise','RetailManager Professional','RetailReady','AccountEdge NE','PACs','Payroll Software','Cashbook','Training & Consulting Services','Training & Consulting Services, Other','Training - All Classroom Training','Training & Consulting Services - Other','MYOB Kounta','Mobile Payments','Cashbook Online','ACE Payroll','IMS Payroll')"
print (orditem_prod_ln_filter)

PRODLN.NAME IN ('AccountEdge','Accounting','AccountingPlus','Additional Bookkeeper','Approved Bookkeeper','AssetManager Pro','Bookkeeper','Business Basics','Certified Consultants','Developer Program','Educators','FirstEdge','JustInvoices','Powerpay','Premier','Premier Enterprise','Publishers','RetailBasics','RetailHospitality','RetailManager','RetailManager Enterprise','RetailManager Professional','RetailReady','AccountEdge NE','PACs','Payroll Software','Cashbook','Training & Consulting Services','Training & Consulting Services, Other','Training - All Classroom Training','Training & Consulting Services - Other','MYOB Kounta','Mobile Payments','Cashbook Online','ACE Payroll','IMS Payroll')


## Order Item Product Part number filter

In [14]:
orditem_prod_part_num_filter = "(PROD.PART_NUM NOT LIKE '%-POR-%' OR PROD.PART_NUM LIKE '%-POR-INV-%')"
print (orditem_prod_part_num_filter)

(PROD.PART_NUM NOT LIKE '%-POR-%' OR PROD.PART_NUM LIKE '%-POR-INV-%')


## S_ORDER_ITEM table ORDITM

1. Restricted the number of rows by limiting orders based on order dates, order status, order item product_type, order item prod line and order item prod part num.

In [15]:
query_S_ORDER_ITM = ("SELECT ORDITEM.ROW_ID ORDITM_ROW_ID, ORDITEM.ADJ_UNIT_PRI ORDITM_ADJ_UNIT_PRI, ORDITEM.NET_PRI ORDITM_NET_PRI, ORDITEM.ASSET_ID ORDITM_ASSET_ID, ORDITEM.ORDER_ID ORDITM_ORDER_ID, ORDITEM.PAR_ORDER_ITEM_ID ORDITM_PAR_ORDER_ITEM_ID, ORDITEM.PROD_ID ORDITM_PROD_ID, ORDITEM.PROMOTION_ID ORDITM_PROMOTION_ID, ORDITEM.OWNER_ACCOUNT_ID ORDITM_OWNER_ACCOUNT_ID, ORDITEM.QTY_REQ ORDITM_QTY_REQ FROM S_ORDER_ITEM ORDITEM INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ORDITEM.ORDER_ID INNER JOIN S_PROD_INT PROD ON ORDITEM.PROD_ID = PROD.ROW_ID AND PROD.TYPE IS NOT NULL INNER JOIN S_PROD_LN PRODLN ON PROD.PR_PROD_LN_ID = PRODLN.ROW_ID WHERE %s AND %s AND %s AND %s AND %s" %(order_dt_filter, order_status_filter, product_price_type_filter, orditem_prod_ln_filter,orditem_prod_part_num_filter))

df_S_ORDER_ITEM = pd.read_sql(query_S_ORDER_ITM, cnxn)
df_S_ORDER_ITEM.head(3)

,ORDITM_ROW_ID,ORDITM_ADJ_UNIT_PRI,ORDITM_NET_PRI,ORDITM_ASSET_ID,ORDITM_ORDER_ID,ORDITM_PAR_ORDER_ITEM_ID,ORDITM_PROD_ID,ORDITM_PROMOTION_ID,ORDITM_OWNER_ACCOUNT_ID,ORDITM_QTY_REQ
0,1-KP1ZHAD,120.000,120.000,None,1-KP1ZHAB,None,1-KP1WPJZ,None,1-KP1YMMI,1.0
1,1-KP31TLS,0.000,0.000,1-KP31BFT,1-KP31TLQ,None,1-11XL7K5,None,1-KP30OAA,1.0
2,1-KP31TLT,90.909,90.909,None,1-KP31TLQ,1-KP31TLS,1-11XL76S,None,1-KP30OAA,1.0


In [16]:
df_S_ORDER_ITEM.count()

ORDITM_ROW_ID               6820
ORDITM_ADJ_UNIT_PRI         5746
ORDITM_NET_PRI              5804
ORDITM_ASSET_ID             2140
ORDITM_ORDER_ID             6820
ORDITM_PAR_ORDER_ITEM_ID    4407
ORDITM_PROD_ID              6820
ORDITM_PROMOTION_ID         1236
ORDITM_OWNER_ACCOUNT_ID     6820
ORDITM_QTY_REQ              6820
dtype: int64

## S_ORDER_ITEM - Parent Order Item PARORDITM
1. Includes Product, Product Line

In [17]:
query_S_ORDER_ITM_PAR_ITEM = ("SELECT ORDITEM.ROW_ID PARORDITM_ORDITEM_ROW_ID, ORDITEM.PAR_ORDER_ITEM_ID PARORDITM_PAR_ORDER_ITEM_ID, ORDITEM.PROD_ID PARORDITM_PROD_ID, ORDITEM.PROMOTION_ID PARORDITM_PROMOTION_ID, PARPROD.PR_PROD_LN_ID PARORDITM_PR_PROD_LN_ID, PARPROD.NAME PARORDITM_PARENT_PROD_NAME, PARPROD.ROW_ID PARORDITM_PARPROD_ROW_ID, PARPROD_LN.NAME PARORDITM_PARENT_PROD_PROD_LINE FROM S_ORDER_ITEM ORDITEM INNER JOIN S_ORDER_ITEM PAR_ORDITEM ON PAR_ORDITEM.ROW_ID = ORDITEM.PAR_ORDER_ITEM_ID left outer join S_PROD_INT PARPROD ON PAR_ORDITEM.PROD_ID = PARPROD.ROW_ID left outer join S_PROD_LN PARPROD_LN ON PARPROD.PR_PROD_LN_ID = PARPROD_LN.ROW_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ORDITEM.ORDER_ID INNER JOIN S_PROD_INT PROD ON ORDITEM.PROD_ID = PROD.ROW_ID AND PROD.TYPE IS NOT NULL INNER JOIN S_PROD_LN PRODLN ON PROD.PR_PROD_LN_ID = PRODLN.ROW_ID WHERE %s AND %s AND %s AND %s AND %s" %(order_dt_filter, order_status_filter, product_price_type_filter, orditem_prod_ln_filter,orditem_prod_part_num_filter))

df_S_ORDER_ITEM_PAR_ITEM = pd.read_sql(query_S_ORDER_ITM_PAR_ITEM, cnxn)
df_S_ORDER_ITEM_PAR_ITEM.head(3)

,PARORDITM_ORDITEM_ROW_ID,PARORDITM_PAR_ORDER_ITEM_ID,PARORDITM_PROD_ID,PARORDITM_PROMOTION_ID,PARORDITM_PR_PROD_LN_ID,PARORDITM_PARENT_PROD_NAME,PARORDITM_PARPROD_ROW_ID,PARORDITM_PARENT_PROD_PROD_LINE
0,1-KP55PTU,1-KP55PTT,1-G5MG,None,1-170ZCT,MYOB AccountEdge Basic - AU,1-11B5SXW,FirstEdge
1,1-KP57B30,1-KP57B2Z,1-13H6RP7,1-LDOGKF,1-170ZCT,MYOB AccountEdge Basic - AU,1-11B5SXW,FirstEdge
2,1-KP57B31,1-KP57B2Z,1-ARAA,1-LDOGKF,1-170ZCT,MYOB AccountEdge Basic - AU,1-11B5SXW,FirstEdge


In [18]:
df_S_ORDER_ITEM_PAR_ITEM.count()

PARORDITM_ORDITEM_ROW_ID           4407
PARORDITM_PAR_ORDER_ITEM_ID        4407
PARORDITM_PROD_ID                  4407
PARORDITM_PROMOTION_ID             1112
PARORDITM_PR_PROD_LN_ID            4407
PARORDITM_PARENT_PROD_NAME         4407
PARORDITM_PARPROD_ROW_ID           4407
PARORDITM_PARENT_PROD_PROD_LINE    4407
dtype: int64

## S_ASSET table - Order Item Asset ORDITMASSET

In [19]:
query_S_ASSET = ("SELECT ASSET.ROW_ID ORDITMASSET_ASSET_ROW_ID, IsNull(CONVERT (VARCHAR (10), REGISTERED_DT, 103),'') ORDITMASSET_REG_DATE, ORDITEM.ROW_ID ORDITMASSET_ORDITEM_ID FROM S_ASSET ASSET INNER JOIN S_ORDER_ITEM ORDITEM ON ORDITEM.ASSET_ID = ASSET.ROW_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ORDITEM.ORDER_ID WHERE %s" %(order_dt_filter))

df_S_ASSET = pd.read_sql(query_S_ASSET, cnxn)
df_S_ASSET.head(3)

,ORDITMASSET_ASSET_ROW_ID,ORDITMASSET_REG_DATE,ORDITMASSET_ORDITEM_ID
0,1-10VB7VQ,13/09/2013,1-L9UQQ98
1,1-10VB7VQ,13/09/2013,1-L34J5H9
2,1-10VB7VQ,13/09/2013,1-KX1PU2F


In [20]:
df_S_ASSET.count()

ORDITMASSET_ASSET_ROW_ID    3184
ORDITMASSET_REG_DATE        3184
ORDITMASSET_ORDITEM_ID      3184
dtype: int64

## S_ORDER table ORD
### Restricted the number of rows by limiting orders based on order dates

In [21]:
query_S_ORDER = ("SELECT ORD.ROW_ID ORD_ORDER_ID, ORD.BU_ID ORD_BU_ID, ORD.ORDER_TYPE_ID ORD_ORDER_TYPE_ID, ORD.BILL_ACCNT_ID ORD_BILL_ACCNT_ID, ORD.ACCNT_ID ORD_ACCNT_ID, ORD.QUOTE_ID ORD_QUOTE_ID, ORD.PROMO_ID ORD_PROMO_ID, ORD.X_PAYMENT_PROFILE_ID ORD_X_PAYMENT_PROFILE_ID, ORD.PR_POSTN_ID ORD_PR_POSTN_ID, ORD.ORDER_DT ORD_ORDER_DT, ORD.STATUS_CD ORD_STATUS_CD FROM S_ORDER ORD WHERE %s" %(order_dt_filter))

df_S_ORDER = pd.read_sql(query_S_ORDER, cnxn)
df_S_ORDER.head(3)

,ORD_ORDER_ID,ORD_BU_ID,ORD_ORDER_TYPE_ID,ORD_BILL_ACCNT_ID,ORD_ACCNT_ID,ORD_QUOTE_ID,ORD_PROMO_ID,ORD_X_PAYMENT_PROFILE_ID,ORD_PR_POSTN_ID,ORD_ORDER_DT,ORD_STATUS_CD
0,1-KOW5PLQ,1-14J-0,0-JFE3,1-J7U4SBC,1-J7U4SBC,None,None,1-KLA005N,0-55RNY,2018-11-30 14:06:17,Pending
1,1-KOW5Q6U,1-14J-0,0-JFE3,1-J7U4SBC,1-J7U4SBC,None,None,1-KLA005N,0-55RNY,2018-11-30 14:16:18,Pending
2,1-KOWIZMM,1-14J-0,0-JFE3,1-J7U4SBC,1-J7U4SBC,None,None,1-KLA005N,0-55RNY,2018-11-30 14:26:16,Pending


In [22]:
df_S_ORDER.count()

ORD_ORDER_ID                23062
ORD_BU_ID                   23062
ORD_ORDER_TYPE_ID           23062
ORD_BILL_ACCNT_ID           23061
ORD_ACCNT_ID                23061
ORD_QUOTE_ID                 1615
ORD_PROMO_ID                    0
ORD_X_PAYMENT_PROFILE_ID    22440
ORD_PR_POSTN_ID             23062
ORD_ORDER_DT                23062
ORD_STATUS_CD               23062
dtype: int64

## S_ORDER_TYPE table ORDTYPE
### This filters the Orders which has the sales type of Sales Order or Web Order.

In [23]:
query_S_ORDER_TYPE = ("SELECT ORDTYPE.ROW_ID ORDTYPE_ID, ORDTYPE.NAME ORDTYPE_NAME, ORD.ROW_ID ORDTYPE_ORDER_ID FROM S_ORDER_TYPE ORDTYPE INNER JOIN S_ORDER ORD ON ORDTYPE.ROW_ID = ORD.ORDER_TYPE_ID WHERE ORDTYPE.NAME IN ('Sales Order', 'Web Order') AND %s" %(order_dt_filter))

df_S_ORDER_TYPE = pd.read_sql(query_S_ORDER_TYPE, cnxn)
df_S_ORDER_TYPE.head(3)

,ORDTYPE_ID,ORDTYPE_NAME,ORDTYPE_ORDER_ID
0,0-JFE3,Web Order,1-KOW5PLQ
1,0-JFE3,Web Order,1-KOW5Q6U
2,0-JFE3,Web Order,1-KOWIZMM


In [24]:
df_S_ORDER_TYPE.count()

ORDTYPE_ID          23009
ORDTYPE_NAME        23009
ORDTYPE_ORDER_ID    23009
dtype: int64

## S_PROD_INT table PROD

In [25]:
query_S_PROD_INT = ("SELECT PROD.ROW_ID PROD_ID, PROD.PRICE_TYPE_CD PROD_PRICE_TYPE, PROD.X_GL_POSTING_GRP PROD_GL_POSTING_GRP, PROD.PART_NUM PROD_PART_NUM, PROD.UOM_CD PROD_UOM, PROD.X_IGNORE_CURR_AGREE_FLG PROD_IGNORE_CURR_AGREE_FLG, PROD.PRICE_TYPE_CD PROD_PRICE_TYPE_CD, PROD.NAME PROD_NAME, PROD.PR_PROD_LN_ID PROD_LN_ID FROM S_PROD_INT PROD WHERE PROD.TYPE IS NOT NULL")

df_S_PROD_INT = pd.read_sql(query_S_PROD_INT, cnxn)
df_S_PROD_INT.head(3)

,PROD_ID,PROD_PRICE_TYPE,PROD_GL_POSTING_GRP,PROD_PART_NUM,PROD_UOM,PROD_IGNORE_CURR_AGREE_FLG,PROD_PRICE_TYPE_CD,PROD_NAME,PROD_LN_ID
0,0-11PCR,None,None,SEBLRSVLP1,Each,None,None,Package,1-1QG2U
1,0-DE5J,One-Time,MANUALS,SEBLRSVWIP1,Each,None,One-Time,Write-In Product,1-AG5FVM
2,04-EPZ0H,One-Time,MAINTAR,Z,Day,N,One-Time,Penalty Charges,1-1QG2U


In [26]:
df_S_PROD_INT.count()

PROD_ID                       15581
PROD_PRICE_TYPE               15580
PROD_GL_POSTING_GRP           15511
PROD_PART_NUM                 15568
PROD_UOM                      15565
PROD_IGNORE_CURR_AGREE_FLG     1585
PROD_PRICE_TYPE_CD            15580
PROD_NAME                     15581
PROD_LN_ID                    15581
dtype: int64

## S_PROD_INT - Order Item Promo Product ORDITMPPROD

In [27]:
query_S_PROD_INT_ORDITEM_PROMO = ("SELECT PROD.ROW_ID ORDITMPPROD_PROD_ID, PROD.PART_NUM ORDITMPPROD_PROD_PART_NUM, PROD.NAME ORDITMPPROD_NAME FROM S_PROD_INT PROD")

df_S_PROD_INT_ORDITEM_PROMO = pd.read_sql(query_S_PROD_INT_ORDITEM_PROMO, cnxn)
df_S_PROD_INT_ORDITEM_PROMO.head(3)

,ORDITMPPROD_PROD_ID,ORDITMPPROD_PROD_PART_NUM,ORDITMPPROD_NAME
0,0-11PCR,SEBLRSVLP1,Package
1,0-DE5J,SEBLRSVWIP1,Write-In Product
2,0-DE5L,SEBLRSVNP1,Customer Note


In [28]:
df_S_PROD_INT_ORDITEM_PROMO.count()

ORDITMPPROD_PROD_ID          18725
ORDITMPPROD_PROD_PART_NUM    18681
ORDITMPPROD_NAME             18725
dtype: int64

## S_PROD_LN TABLE PRODLN

In [29]:
query_S_PROD_LN = ("SELECT PRODLN.ROW_ID PRODLN_ROW_ID, PRODLN.NAME PRODLN_NAME FROM S_PROD_LN PRODLN")

df_S_PROD_LN = pd.read_sql(query_S_PROD_LN, cnxn)
df_S_PROD_LN.head(3)

,PRODLN_ROW_ID,PRODLN_NAME
0,1-740CD6O,ACE Payroll
1,1-CPEJ4C,AD Other Family
2,1-CPEJ4A,AE Client Accounting Family


In [30]:
df_S_PROD_LN.count()

PRODLN_ROW_ID    232
PRODLN_NAME      232
dtype: int64

## S_INVOICE_ITEM - Order Item Invoice ORDITMINV

In [31]:
query_S_INVOICE_ITEM = ("SELECT ITEM.ROW_ID ORDITMINV_ORDITEM_ROW_ID, INVOICE_ID ORDITMINV_INVOICE_ID, INV_ITEM.ROW_ID ORDITMINV_ITEM_ID FROM S_INVOICE_ITEM INV_ITEM INNER JOIN S_ORDER_ITEM ITEM ON ITEM.ROW_ID = INV_ITEM.ORDER_ITEM_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ITEM.ORDER_ID WHERE %s" %(order_dt_filter))

df_S_INVOICE_ITEM = pd.read_sql(query_S_INVOICE_ITEM, cnxn)
df_S_INVOICE_ITEM.head(3)

,ORDITMINV_ORDITEM_ROW_ID,ORDITMINV_INVOICE_ID,ORDITMINV_ITEM_ID
0,1-KP1ZHAD,1-KOOSCGD,1-KOOSCGJ
1,1-KP234C6,1-KOOSCT2,1-KOOSCT8
2,1-KP234C7,1-KOOSCT2,1-KOOSCTK


In [32]:
df_S_INVOICE_ITEM.count()

ORDITMINV_ORDITEM_ROW_ID    5241
ORDITMINV_INVOICE_ID        5241
ORDITMINV_ITEM_ID           5241
dtype: int64

## S_INVOICE TABLE INV
### Restricted the number of rows by limiting invoices based on order dates

In [33]:
query_S_INVOICE = ("SELECT ORD_ITEM.ROW_ID INV_ORDITEM_ROW_ID, INV.ROW_ID AS INV_ROW_ID, INV.INVC_NUM AS INV_NUMBER, IsNull(CONVERT (VARCHAR (10), DATEADD(hh, 10, INV.INVC_DT), 103),'') AS INV_DATE FROM S_INVOICE INV INNER JOIN S_INVOICE_ITEM INV_ITEM ON INV_ITEM.INVOICE_ID = INV.ROW_ID INNER JOIN S_ORDER_ITEM ORD_ITEM ON ORD_ITEM.ROW_ID = INV_ITEM.ORDER_ITEM_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ORD_ITEM.ORDER_ID WHERE %s" %(order_dt_filter))

df_S_INVOICE = pd.read_sql(query_S_INVOICE, cnxn)
df_S_INVOICE.head(3)

,INV_ORDITEM_ROW_ID,INV_ROW_ID,INV_NUMBER,INV_DATE
0,1-KP1ZHAD,1-KOOSCGD,1-45028468237,01/12/2018
1,1-KP234C6,1-KOOSCT2,1-45028468694,01/12/2018
2,1-KP234C7,1-KOOSCT2,1-45028468694,01/12/2018


In [34]:
df_S_INVOICE.count()

INV_ORDITEM_ROW_ID    5241
INV_ROW_ID            5241
INV_NUMBER            5241
INV_DATE              5241
dtype: int64

## S_PROMO_ITEM TABLE PROMO

In [35]:
query_S_PROMO_ITEM = ("SELECT PROMOTION_ID AS PROMO_ID, count(*) AS PROMO_UPG_COUNT from S_PROM_ITEM inner join S_PROD_INT ON S_PROM_ITEM.PROD_ID = S_PROD_INT.ROW_ID where S_PROD_INT.TYPE = 'Upgrade' group by PROMOTION_ID")

df_S_PROMO_ITEM = pd.read_sql(query_S_PROMO_ITEM, cnxn)
df_S_PROMO_ITEM.head(3)

,PROMO_ID,PROMO_UPG_COUNT
0,1-IBPK1,2
1,1-16ODG9G,1
2,1-AV7I,1


In [36]:
df_S_PROMO_ITEM.count()

PROMO_ID           1691
PROMO_UPG_COUNT    1691
dtype: int64

# S_ORG_EXT TABLE - BILLING ACCOUNT BILLACC

In [37]:
query_S_ORG_EXT_BILL = ("SELECT ORD.ROW_ID BILLACC_ORD_ROW_ID, ACCNT_BILL.ROW_ID BILLACC_ROW_ID, ACCNT_BILL.OU_TYPE_CD BILLACC_OU_TYPE_CD, ACCNT_BILL.INTEGRATION_ID BILLACC_INTEGRATION_ID, ACCNT_BILL.X_SEGMENT BILLACC_X_SEGMENT, ACCNT_BILL.X_PROD_EVNGLIST_ID BILLACC_EVNGLIST_ID, ACCNT_BILL.X_DIVISION BILLACC_DIVISION, ACCNT_BILL.CREATED BILLACC_CREATED, ACCNT_BILL.NAME BILLACC_NAME, ACCNT_BILL.PR_POSTN_ID BILLACC_PR_POSTN_ID,ACC_EVANG.LOGIN BILLACC_LOGIN FROM S_ORG_EXT ACCNT_BILL INNER JOIN S_ORDER ORD ON ORD.BILL_ACCNT_ID = ACCNT_BILL.ROW_ID LEFT OUTER JOIN S_USER ACC_EVANG ON ACCNT_BILL.X_PROD_EVNGLIST_ID  = ACC_EVANG.ROW_ID WHERE %s" %(order_dt_filter))

df_S_ORG_EXT_BILL = pd.read_sql(query_S_ORG_EXT_BILL, cnxn)
df_S_ORG_EXT_BILL.head(3)

,BILLACC_ORD_ROW_ID,BILLACC_ROW_ID,BILLACC_OU_TYPE_CD,BILLACC_INTEGRATION_ID,BILLACC_X_SEGMENT,BILLACC_EVNGLIST_ID,BILLACC_DIVISION,BILLACC_CREATED,BILLACC_NAME,BILLACC_PR_POSTN_ID,BILLACC_LOGIN
0,1-L29I9H8,1-10073XV,Customer,8209598,Prime,None,Accountants Division,2013-09-09 12:27:53,Tester's Partner Business,0-55RNY,None
1,1-L2999AE,1-10073XV,Customer,8209598,Prime,None,Accountants Division,2013-09-09 12:27:53,Tester's Partner Business,0-55RNY,None
2,1-L4VBTIU,1-100CPR4,Customer,8209631,Growing Business,None,Business Division,2013-09-09 12:33:35,ANZ,0-55RNY,None


In [38]:
df_S_ORG_EXT_BILL.count()

BILLACC_ORD_ROW_ID        23061
BILLACC_ROW_ID            23061
BILLACC_OU_TYPE_CD        23061
BILLACC_INTEGRATION_ID    23061
BILLACC_X_SEGMENT         23061
BILLACC_EVNGLIST_ID          13
BILLACC_DIVISION          23061
BILLACC_CREATED           23061
BILLACC_NAME              23061
BILLACC_PR_POSTN_ID       23061
BILLACC_LOGIN                13
dtype: int64

## S_ORG_EXT table - ORDER_ACCNT ORDACC

In [39]:
query_S_ORG_EXT_ORD_ACCNT = ("SELECT ORD.ROW_ID ORDACC_ORDER_ID, ORD_ACCNT.ROW_ID AS ORDACC_ID, ORD_ACCNT.OU_TYPE_CD AS ORDACC_OU_TYPE_CD_INT FROM S_ORG_EXT ORD_ACCNT INNER JOIN S_ORDER ORD ON ORD.ACCNT_ID = ORD_ACCNT.ROW_ID WHERE %s" %(order_dt_filter))

df_S_ORG_EXT_ORD_ACCNT = pd.read_sql(query_S_ORG_EXT_ORD_ACCNT, cnxn)
df_S_ORG_EXT_ORD_ACCNT.head(3)

,ORDACC_ORDER_ID,ORDACC_ID,ORDACC_OU_TYPE_CD_INT
0,1-KW6EQ1A,1-WYWR17,Business Partner
1,1-KW8D6P9,1-WYWR17,Business Partner
2,1-KW9348B,1-WYWR17,Business Partner


In [40]:
df_S_ORG_EXT_ORD_ACCNT.count()

ORDACC_ORDER_ID          23061
ORDACC_ID                23061
ORDACC_OU_TYPE_CD_INT    23061
dtype: int64

## S_ASSET TABLE - PARENT ORDER ITEM ASSET PARORDITMAST
1. PRIMARY AGREE ITEM

In [41]:
query_S_ASSET_PAR_ORD_ITEM = ("SELECT ORDITEM.ROW_ID PARORDITMAST_ORDITEM_ROW_ID, PAR_ORD_ITEM_ASSET.ROW_ID PARORDITMAST_ROW_ID, PAR_ORD_ITEM_ASSET.SERIAL_NUM AS PARORDITMAST_SERIAL_NUM, PAR_ORD_ITEM_ASSET.X_PR_AGREE_ITEM_ID PARORDITMAST_PR_AGREE_ITEM_ID, PRIMARY_ITEM.DOC_AGREE_ID PARORDITMAST_PRIMARY_AGR_ID,CASE WHEN BUREAU_ASSET.ROW_ID IS NULL THEN 'N' ELSE 'Y' END AS PARORDITMAST_BUREAU_FLG FROM S_ASSET PAR_ORD_ITEM_ASSET INNER JOIN S_ORDER_ITEM PAR_ORD_ITEM ON PAR_ORD_ITEM.ASSET_ID = PAR_ORD_ITEM_ASSET.ROW_ID INNER JOIN S_ORDER_ITEM ORDITEM ON ORDITEM.PAR_ORDER_ITEM_ID = PAR_ORD_ITEM.ROW_ID LEFT OUTER JOIN S_AGREE_ITEM PRIMARY_ITEM ON PRIMARY_ITEM.ROW_ID = PAR_ORD_ITEM_ASSET.X_PR_AGREE_ITEM_ID LEFT OUTER JOIN S_ASSET_REL BUREAU_ASSET ON PAR_ORD_ITEM_ASSET.ROW_ID = BUREAU_ASSET.ASSET_ID AND BUREAU_ASSET.RELATION_TYPE_CD = 'Relates To' INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ORDITEM.ORDER_ID WHERE %s" %(order_dt_filter))

df_S_ASSET_PAR_ORD_ITEM_ASSET = pd.read_sql(query_S_ASSET_PAR_ORD_ITEM, cnxn)
df_S_ASSET_PAR_ORD_ITEM_ASSET.head(3)

,PARORDITMAST_ORDITEM_ROW_ID,PARORDITMAST_ROW_ID,PARORDITMAST_SERIAL_NUM,PARORDITMAST_PR_AGREE_ITEM_ID,PARORDITMAST_PRIMARY_AGR_ID,PARORDITMAST_BUREAU_FLG
0,1-LGAHZEP,1-10VB7VQ,None,None,None,N
1,1-L9UQQ1H,1-10VB7VQ,None,None,None,N
2,1-L34ISM8,1-10VB7VQ,None,None,None,N


In [42]:
df_S_ASSET_PAR_ORD_ITEM_ASSET.count()

PARORDITMAST_ORDITEM_ROW_ID      6358
PARORDITMAST_ROW_ID              6358
PARORDITMAST_SERIAL_NUM          6278
PARORDITMAST_PR_AGREE_ITEM_ID    4385
PARORDITMAST_PRIMARY_AGR_ID      4385
PARORDITMAST_BUREAU_FLG          6358
dtype: int64

## S_ORG_EXT TABLE - LEGAL ENTITY

In [43]:
query_S_ORG_EXT_LE = ("SELECT ROW_ID LEGAL_ENTITY_ID FROM S_ORG_EXT WHERE ROW_ID IN ('1-14J-0','1-14J-1')")

df_S_ORG_EXT_LE = pd.read_sql(query_S_ORG_EXT_LE, cnxn)
df_S_ORG_EXT_LE.head(3)

,LEGAL_ENTITY_ID
0,1-14J-0
1,1-14J-1


In [44]:
df_S_ORG_EXT_LE.count()

LEGAL_ENTITY_ID    2
dtype: int64

## S_AGREE_ITEM - Order Agreement Item, use RANK TO GET 1 AGREEMENT ITEM ONLY, Agree Referred by Account, Evangelist. AGRITMEVG

1. NEED TO DO THE RANK FILTER AFTER THE FIRST DF IS FORMED AS WE CANNOT ADD WINDOW FUNCTION IN THE WHERE CLAUSE.
2. Obtain the previous agreement item using the X_RENEWED_BY_ITEM_ID.


In [45]:
query_S_AGREE_ITEM_ORD_AGREE = ("SELECT AGR.ORDER_ID AGRITMEVG_ORDER_ID, ITEM.ROW_ID AGRITMEVG_ITEM_ID, ITEM.ASSET_ID AGRITMEVG_ASSET_ID, ITEM.PROD_INT_ID AGRITMEVG_ITEM_PROD_ID,RANK() OVER(PARTITION BY AGR.ORDER_ID,ITEM.ASSET_ID,ITEM.PROD_INT_ID ORDER BY AGR.CREATED DESC) AGRITMEVG_RANKS,RANK() OVER(PARTITION BY ITEM.DOC_AGREE_ID,ITEM.PROD_INT_ID,ITEM.ASSET_ID ORDER BY ITEM.ROW_ID) AGRITMEVG_RANKS2, ITEM.DOC_AGREE_ID AGRITMEVG_ITEM_DOC_AGREE_ID, ITEM.X_USED_BY_ASSET_ID AGRITMEVG_AGREE_ITEM_CDF_ROW_ID, RENEWED_ITEM.ROW_ID AGRITMEVG_RENEWED_ITEM_ROW_ID, ORDITEM.ROW_ID AGRITMEVG_ORDITEM_ROW_ID from S_AGREE_ITEM ITEM inner join S_DOC_AGREE AGR ON ITEM.DOC_AGREE_ID = AGR.ROW_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = AGR.ORDER_ID INNER JOIN S_ORDER_ITEM ORDITEM ON ORD.ROW_ID = ORDITEM.ORDER_ID AND ORDITEM.PROD_ID = ITEM.PROD_INT_ID INNER JOIN S_ORDER_ITEM PAR_ORD_ITEM ON ORDITEM.PAR_ORDER_ITEM_ID = PAR_ORD_ITEM.ROW_ID INNER JOIN S_ASSET PAR_ORD_ITEM_ASSET ON PAR_ORD_ITEM_ASSET.ROW_ID = ITEM.ASSET_ID LEFT OUTER JOIN S_AGREE_ITEM RENEWED_ITEM ON RENEWED_ITEM.X_RENEWED_BY_ITEM_ID = ITEM.ROW_ID WHERE %s" %(order_dt_filter))

df_S_AGREE_ITEM_ORD_AGREE = pd.read_sql(query_S_AGREE_ITEM_ORD_AGREE, cnxn)

# Filter data based on RANKS as in original query
df_S_AGREE_ITEM_ORD_AGREE = df_S_AGREE_ITEM_ORD_AGREE.loc[(df_S_AGREE_ITEM_ORD_AGREE['AGRITMEVG_RANKS'] == 1) & (df_S_AGREE_ITEM_ORD_AGREE['AGRITMEVG_RANKS2'] == 1)]
df_S_AGREE_ITEM_ORD_AGREE.head(3)

,AGRITMEVG_ORDER_ID,AGRITMEVG_ITEM_ID,AGRITMEVG_ASSET_ID,AGRITMEVG_ITEM_PROD_ID,AGRITMEVG_RANKS,AGRITMEVG_RANKS2,AGRITMEVG_ITEM_DOC_AGREE_ID,AGRITMEVG_AGREE_ITEM_CDF_ROW_ID,AGRITMEVG_RENEWED_ITEM_ROW_ID,AGRITMEVG_ORDITEM_ROW_ID
0,1-KP1SS9G,1-KP1TOF1,1-KP1T1GR,1-1203J1J,1,1,1-KP1TOEL,None,None,1-KP1SS9J
1,1-KP1SS9G,1-KP1U29X,1-KP1T1GR,1-AREE,1,1,1-KP1U29H,1-KOOOPYG,None,1-KP1T104
2,1-KP1W521,1-KP1WJW9,1-KP1W5I5,1-1203J1J,1,1,1-KP1WJVT,None,None,1-KP1W524


In [46]:
df_S_AGREE_ITEM_ORD_AGREE.count()

AGRITMEVG_ORDER_ID                 5031
AGRITMEVG_ITEM_ID                  5031
AGRITMEVG_ASSET_ID                 5031
AGRITMEVG_ITEM_PROD_ID             5031
AGRITMEVG_RANKS                    5031
AGRITMEVG_RANKS2                   5031
AGRITMEVG_ITEM_DOC_AGREE_ID        5031
AGRITMEVG_AGREE_ITEM_CDF_ROW_ID      47
AGRITMEVG_RENEWED_ITEM_ROW_ID       809
AGRITMEVG_ORDITEM_ROW_ID           5031
dtype: int64

## S_DOC_AGREE TABLE - INITIAL AGREEMENT, using RANK to get 1 Agree Item only INITAGR
1. This will join need to join with the data from section: S_AGREE_ITEM - Order Agreement Item, use RANK TO GET 1 AGREEMENT ITEM ONLY. One is to provide the latest agreement and other is to provide the initial agreement.

In [47]:
query_initial_agree = ("SELECT DISTINCT INIT_AGREE.EFF_START_DT INITAGR_EFF_START_DT,INIT_AGREE.ORDER_ID INITAGR_ORDER_ID,INIT_AGREE.ROW_ID INITAGR_AGREE_ROW_ID,INIT_AGREE.AGREE_NUM INITAGR_NUM,S_ASSET.ROW_ID INITAGR_ASSET_ROW_ID,RANK() OVER(PARTITION BY INIT_AGREE.ORDER_ID, S_ASSET.ROW_ID ORDER BY INIT_AGREE.EFF_START_DT ASC) INITAGR_RANKS_INIT FROM S_DOC_AGREE INIT_AGREE INNER JOIN S_AGREE_ITEM ON S_AGREE_ITEM.DOC_AGREE_ID = INIT_AGREE.ROW_ID INNER JOIN S_ASSET ON S_ASSET.ROW_ID = S_AGREE_ITEM.ASSET_ID INNER JOIN S_PROD_INT ON S_AGREE_ITEM.PROD_INT_ID = S_PROD_INT.ROW_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = INIT_AGREE.ORDER_ID WHERE DATEDIFF(day, INIT_AGREE.EFF_START_DT, INIT_AGREE.EFF_END_DT) < 183 AND INIT_AGREE.AGREE_CD != 'Introductory Support' AND INIT_AGREE.ORDER_ID IS NOT NULL AND S_PROD_INT.PRICE_TYPE_CD ='One-Time' AND %s" %(order_dt_filter))

df_S_AGREE_ITEM_ORD_INITIAL_AGREE = pd.read_sql(query_initial_agree, cnxn)

df_S_AGREE_ITEM_ORD_INITIAL_AGREE.head(3)

# join with the df_S_AGREE_ITEM_ORD_AGREE
df_S_AGREE_ITEM_ORD_INITIAL_AGREE = pd.merge(df_S_AGREE_ITEM_ORD_INITIAL_AGREE, df_S_AGREE_ITEM_ORD_AGREE, how='left', left_on=['INITAGR_ASSET_ROW_ID'], right_on=['AGRITMEVG_ASSET_ID'])

# Only rows that are not the same as the current agreement and its the earilest.
df_S_AGREE_ITEM_ORD_INITIAL_AGREE = df_S_AGREE_ITEM_ORD_INITIAL_AGREE.loc[(df_S_AGREE_ITEM_ORD_INITIAL_AGREE['INITAGR_RANKS_INIT'] == 1) & (df_S_AGREE_ITEM_ORD_INITIAL_AGREE['INITAGR_AGREE_ROW_ID'] != df_S_AGREE_ITEM_ORD_INITIAL_AGREE['AGRITMEVG_ITEM_DOC_AGREE_ID'])]
df_S_AGREE_ITEM_ORD_INITIAL_AGREE.head(3)

,INITAGR_EFF_START_DT,INITAGR_ORDER_ID,INITAGR_AGREE_ROW_ID,INITAGR_NUM,INITAGR_ASSET_ROW_ID,INITAGR_RANKS_INIT,AGRITMEVG_ORDER_ID,AGRITMEVG_ITEM_ID,AGRITMEVG_ASSET_ID,AGRITMEVG_ITEM_PROD_ID,AGRITMEVG_RANKS,AGRITMEVG_RANKS2,AGRITMEVG_ITEM_DOC_AGREE_ID,AGRITMEVG_AGREE_ITEM_CDF_ROW_ID,AGRITMEVG_RENEWED_ITEM_ROW_ID,AGRITMEVG_ORDITEM_ROW_ID
1,2018-11-30 18:44:20,1-KP496DK,1-KP49NUL,1-45054470397,1-KP49BPN,1,1-KP496DK,1-KP49TE1,1-KP49BPN,1-ARQ2,1,1,1-KP49TDL,None,None,1-KP49B9C
2,2018-11-30 18:44:20,1-KP496DK,1-KP49NUL,1-45054470397,1-KP49BPN,1,1-KP4A0L2,1-KP4BH9L,1-KP49BPN,1-ARQ2,1,1,1-KP4BH95,None,None,1-KP4A0L5
3,2018-11-30 18:44:20,1-KP496DK,1-KP49NUL,1-45054470397,1-KP49BPN,1,1-KP4A0L2,1-KP4BH9L,1-KP49BPN,1-ARQ2,1,1,1-KP4BH95,None,None,1-KP4A0L6


In [48]:
df_S_AGREE_ITEM_ORD_INITIAL_AGREE.count()

INITAGR_EFF_START_DT               294
INITAGR_ORDER_ID                   294
INITAGR_AGREE_ROW_ID               294
INITAGR_NUM                        294
INITAGR_ASSET_ROW_ID               294
INITAGR_RANKS_INIT                 294
AGRITMEVG_ORDER_ID                 294
AGRITMEVG_ITEM_ID                  294
AGRITMEVG_ASSET_ID                 294
AGRITMEVG_ITEM_PROD_ID             294
AGRITMEVG_RANKS                    294
AGRITMEVG_RANKS2                   294
AGRITMEVG_ITEM_DOC_AGREE_ID        294
AGRITMEVG_AGREE_ITEM_CDF_ROW_ID      0
AGRITMEVG_RENEWED_ITEM_ROW_ID       54
AGRITMEVG_ORDITEM_ROW_ID           294
dtype: int64

## S_POSTN Account Manager position contact ACCMGR

In [49]:
query_S_POSTN_ACCT_MGR = ("SELECT ORD.ROW_ID ACCMGR_ORDER_ID, ACCNT_MANAGER_CON.FST_NAME + ' ' +  ACCNT_MANAGER_CON.LAST_NAME AS ACCMGR_NAME FROM S_POSTN ACCOUNT_MANAGER_POS INNER JOIN S_ORG_EXT ACC ON ACC.PR_POSTN_ID = ACCOUNT_MANAGER_POS.ROW_ID INNER JOIN S_CONTACT ACCNT_MANAGER_CON ON ACCNT_MANAGER_CON.ROW_ID = ACCOUNT_MANAGER_POS.PR_EMP_ID INNER JOIN S_ORDER ORD ON ORD.BILL_ACCNT_ID = ACC.ROW_ID WHERE %s" %(order_dt_filter))

df_S_POSTN_ACCT_MGR = pd.read_sql(query_S_POSTN_ACCT_MGR, cnxn)
df_S_POSTN_ACCT_MGR.head(3)

,ACCMGR_ORDER_ID,ACCMGR_NAME
0,1-L27UV4O,Siebel Administrator
1,1-L298WOI,Siebel Administrator
2,1-L29944R,Siebel Administrator


In [50]:
df_S_POSTN_ACCT_MGR.count()

ACCMGR_ORDER_ID    23062
ACCMGR_NAME        23062
dtype: int64

## S_DOC_QUOTE table QUOTE

In [51]:
query_S_DOC_QUOTE = ("SELECT QUOTE.ROW_ID QUOTE_ID, ORD.ROW_ID QUOTE_ORD_ROW_ID, QUOTE.SALES_REP_POSTN_ID QUOTE_SALES_REP_POSTN_ID, QUOTE.X_REF_ACCT_ID QUOTE_X_REF_ACCT_ID, QUOTE.X_PROD_EVNGLIST_ID QUOTE_PROD_EVNGLIST_ID, QUOTE.CREATED_BY QUOTE_CREATED_BY, QUOTE.X_REF_CON_ID QUOTE_REF_CON_ID, QUOTE.QUOTE_NUM QUOTE_NUM, QUOTE.X_COMM_OPTION QUOTE_X_COMM_OPTION FROM S_DOC_QUOTE QUOTE INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID WHERE %s" %(order_dt_filter))

df_S_DOC_QUOTE = pd.read_sql(query_S_DOC_QUOTE, cnxn)
df_S_DOC_QUOTE.head(3)

,QUOTE_ID,QUOTE_ORD_ROW_ID,QUOTE_SALES_REP_POSTN_ID,QUOTE_X_REF_ACCT_ID,QUOTE_PROD_EVNGLIST_ID,QUOTE_CREATED_BY,QUOTE_REF_CON_ID,QUOTE_NUM,QUOTE_X_COMM_OPTION
0,1-KP58J6E,1-KP58JOY,0-55RNY,None,None,1-1PIO0,None,1-KP58J6E,None
1,1-KPZB9P7,1-KPZA6Z7,1-1PENB,None,None,1-1GUJ-651,None,1-KPZB9P7,None
2,1-KPZELMF,1-KPZDP4C,1-1PENB,None,None,1-1GUJ-651,None,1-KPZELMF,None


In [52]:
df_S_DOC_QUOTE.count()

QUOTE_ID                    1615
QUOTE_ORD_ROW_ID            1615
QUOTE_SALES_REP_POSTN_ID    1615
QUOTE_X_REF_ACCT_ID           53
QUOTE_PROD_EVNGLIST_ID        34
QUOTE_CREATED_BY            1615
QUOTE_REF_CON_ID              53
QUOTE_NUM                   1615
QUOTE_X_COMM_OPTION           52
dtype: int64

## S_POSTN Quote Sales Rep position, contact, team, Division QUOTEREP

In [53]:
query_S_POSTN_QUOTE_SALES_REP = ("SELECT ORD.ROW_ID QUOTEREP_ORD_ROW_ID, POS.NAME QUOTEREP_SALES_REP_POSITION, DIV.NAME QUOTEREP_TEAM, QUOTE_SALES_REP.FST_NAME + ' ' + QUOTE_SALES_REP.LAST_NAME QUOTEREP_SALES_REP,TEAM.NAME QUOTEREP_SALES_REP_TEAM FROM S_POSTN POS INNER JOIN S_DOC_QUOTE QUOTE ON QUOTE.SALES_REP_POSTN_ID = POS.ROW_ID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID LEFT OUTER JOIN S_ORG_EXT DIV ON POS.OU_ID = DIV.ROW_ID left outer join S_CONTACT QUOTE_SALES_REP ON QUOTE_SALES_REP.ROW_ID = POS.PR_EMP_ID left outer join S_ORG_EXT TEAM ON TEAM.PAR_ROW_ID = POS.OU_ID WHERE %s" %(order_dt_filter))

df_S_POSTN_QUOTE_SALES_REP = pd.read_sql(query_S_POSTN_QUOTE_SALES_REP, cnxn)
df_S_POSTN_QUOTE_SALES_REP.head(3)

,QUOTEREP_ORD_ROW_ID,QUOTEREP_SALES_REP_POSITION,QUOTEREP_TEAM,QUOTEREP_SALES_REP,QUOTEREP_SALES_REP_TEAM
0,1-KPMN85Q,Siebel Administrator,Siebel Administration,Siebel Administrator,Siebel Administration
1,1-KQYH94D,Siebel Administrator,Siebel Administration,Siebel Administrator,Siebel Administration
2,1-KQZGRAJ,Siebel Administrator,Siebel Administration,Siebel Administrator,Siebel Administration


In [54]:
df_S_POSTN_QUOTE_SALES_REP.count()

QUOTEREP_ORD_ROW_ID            1615
QUOTEREP_SALES_REP_POSITION    1615
QUOTEREP_TEAM                  1615
QUOTEREP_SALES_REP             1615
QUOTEREP_SALES_REP_TEAM        1615
dtype: int64

## S_ORG_EXT - QUOTE Referred by Account QUOTEREFACC
1. Referred by Account Manger Contact.

In [55]:
query_S_AGREE_ITEM_RENEWED_ITEM = ("SELECT ORD.ROW_ID QUOTEREFACC_ORD_ROW_ID, REF.ROW_ID QUOTEREFACC_ID, REF.PR_POSTN_ID QUOTEREFACC_POSTN_ID, ISNULL(REF.OU_TYPE_CD,'NA') QUOTEREFACC_OU_TYPE_CD, REF.INTEGRATION_ID QUOTEREFACC_CLIENT_ID, REF.X_DIVISION QUOTEREFACC_DIVISION, REF.X_SEGMENT QUOTEREFACC_SEGMENT, REF.NAME QUOTEREFACC_NAME, REF_ACCOUNT_MANAGER_CON.FST_NAME + ' ' +  REF_ACCOUNT_MANAGER_CON.LAST_NAME AS QUOTEREFACC_ACCOUNT_MANAGER FROM S_ORG_EXT REF INNER JOIN S_DOC_QUOTE QUOTE ON QUOTE.X_REF_ACCT_ID = REF.ROW_ID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID LEFT OUTER JOIN S_POSTN REF_ACCOUNT_MANAGER_POS ON REF_ACCOUNT_MANAGER_POS.ROW_ID = REF.PR_POSTN_ID LEFT OUTER JOIN S_CONTACT REF_ACCOUNT_MANAGER_CON ON REF_ACCOUNT_MANAGER_CON.ROW_ID = REF_ACCOUNT_MANAGER_POS.PR_EMP_ID WHERE %s" %(order_dt_filter))

df_S_AGREE_ITEM_RENEWED_ITEM = pd.read_sql(query_S_AGREE_ITEM_RENEWED_ITEM, cnxn)
df_S_AGREE_ITEM_RENEWED_ITEM.head(3)

,QUOTEREFACC_ORD_ROW_ID,QUOTEREFACC_ID,QUOTEREFACC_POSTN_ID,QUOTEREFACC_OU_TYPE_CD,QUOTEREFACC_CLIENT_ID,QUOTEREFACC_DIVISION,QUOTEREFACC_SEGMENT,QUOTEREFACC_NAME,QUOTEREFACC_ACCOUNT_MANAGER
0,1-KRQ2NJS,1-1D6KRQJ,1-40U-11,Customer,8308248,Business Division,Growing Business,XSEOAKJIHNNQXWQQHEITWLCIXAWBYDOBINUBTHQE,Anoop Chandran
1,1-KT1GUY7,1-1T9OMOC,1-GLY-7,Business Partner,8321915,Enterprise Division,Enterprise Business,Advanced Partner Pty Ltd,Hayley Punter
2,1-KT1ZHU7,1-1T9OMOC,1-GLY-7,Business Partner,8321915,Enterprise Division,Enterprise Business,Advanced Partner Pty Ltd,Hayley Punter


In [56]:
df_S_AGREE_ITEM_RENEWED_ITEM.count()

QUOTEREFACC_ORD_ROW_ID         53
QUOTEREFACC_ID                 53
QUOTEREFACC_POSTN_ID           53
QUOTEREFACC_OU_TYPE_CD         53
QUOTEREFACC_CLIENT_ID          53
QUOTEREFACC_DIVISION           53
QUOTEREFACC_SEGMENT            53
QUOTEREFACC_NAME               53
QUOTEREFACC_ACCOUNT_MANAGER    53
dtype: int64

## S_USER - QUOTE PROD EVNGLIST FROM ORDER. QUOTEEVN

In [57]:
query_S_USER_QUOTE_EVANG = ("SELECT ORD.ROW_ID QUOTEEVN_ORD_ROW_ID, EVANG.LOGIN QUOTEEVN_LOGIN FROM S_USER EVANG INNER JOIN S_DOC_QUOTE QUOTE ON QUOTE.X_PROD_EVNGLIST_ID = EVANG.ROW_ID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID WHERE %s" %(order_dt_filter))

df_S_USER_QUOTE_EVANG = pd.read_sql(query_S_USER_QUOTE_EVANG, cnxn)
df_S_USER_QUOTE_EVANG.head(3)

,QUOTEEVN_ORD_ROW_ID,QUOTEEVN_LOGIN
0,1-KPMN85Q,SIT.ADSCH7@MYOBTEST.NET
1,1-KQB2U67,SIT.ADSCH7@MYOBTEST.NET
2,1-KQZGRAJ,SIT.ADSCH7@MYOBTEST.NET


In [58]:
df_S_USER_QUOTE_EVANG.count()

QUOTEEVN_ORD_ROW_ID    34
QUOTEEVN_LOGIN         34
dtype: int64

## S_CONTACT - Quote Creater contact QUOTECTR

In [59]:
query_S_CONTACT_Quote_Creater = ("SELECT ORD.ROW_ID QUOTECTR_ORD_ROW_ID, QUOTE.ROW_ID QUOTECTR_QUOTE_ROW_ID, QUOTE_CREATOR.FST_NAME + ' ' + QUOTE_CREATOR.LAST_NAME AS QUOTECTR_CREATOR FROM S_CONTACT QUOTE_CREATOR INNER JOIN S_DOC_QUOTE QUOTE ON QUOTE.CREATED_BY = QUOTE_CREATOR.ROW_ID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID WHERE %s" %(order_dt_filter))
                                 
df_S_CONTACT_Quote_Creater = pd.read_sql(query_S_CONTACT_Quote_Creater, cnxn)
df_S_CONTACT_Quote_Creater.head(3)

,QUOTECTR_ORD_ROW_ID,QUOTECTR_QUOTE_ROW_ID,QUOTECTR_CREATOR
0,1-KPMN85Q,1-KPMN3LW,Siebel Administrator
1,1-KQB2U67,1-KQB2RAN,Siebel Administrator
2,1-KQYH94D,1-KQYH59R,Siebel Administrator


In [60]:
df_S_CONTACT_Quote_Creater.count()

QUOTECTR_ORD_ROW_ID      1615
QUOTECTR_QUOTE_ROW_ID    1615
QUOTECTR_CREATOR         1615
dtype: int64

## S_POSTN - Quote Creater Team QUOTECTRT
1. The team of the Quote creater, if there is more than one team exists, pick the team with the earliest created date 

In [61]:
query_S_POSTN_QUOTE_CREATOR_TEAM = ("SELECT QUOTE_CREATOR_TEAM.NAME AS QUOTECTRT_TEAM, QUOTE.ROW_ID QUOTECTRT_QUOTE_ID, ORD.ROW_ID QUOTECTRT_ORD_ROW_ID, RANK() OVER(PARTITION BY QUOTE.ROW_ID, ORD.ROW_ID ORDER BY QUOTE_CREATOR_POSTN.CREATED) QUOTECTRT_POSTN_RANK FROM S_DOC_QUOTE QUOTE INNER JOIN S_CONTACT QUOTE_CREATOR ON QUOTE_CREATOR.ROW_ID = QUOTE.CREATED_BY INNER JOIN S_POSTN QUOTE_CREATOR_POSTN ON QUOTE_CREATOR_POSTN.PR_EMP_ID = QUOTE_CREATOR.ROW_ID INNER JOIN S_ORG_EXT QUOTE_CREATOR_TEAM ON QUOTE_CREATOR_POSTN.OU_ID = QUOTE_CREATOR_TEAM.ROW_ID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID WHERE %s" % (order_dt_filter))
                                 
df_S_CONTACT_QUOTE_CREATOR_TEAM = pd.read_sql(query_S_POSTN_QUOTE_CREATOR_TEAM, cnxn)
df_S_CONTACT_QUOTE_CREATOR_TEAM = df_S_CONTACT_QUOTE_CREATOR_TEAM.loc[(df_S_CONTACT_QUOTE_CREATOR_TEAM['QUOTECTRT_POSTN_RANK'] == 1)]
df_S_CONTACT_QUOTE_CREATOR_TEAM.head(3)

,QUOTECTRT_TEAM,QUOTECTRT_QUOTE_ID,QUOTECTRT_ORD_ROW_ID,QUOTECTRT_POSTN_RANK
0,Default Organization,1-KO6A79R,1-KP2N1OU,1
1,Default Organization,1-KP2T2X9,1-KP2TAIQ,1
2,Default Organization,1-KP2T5TZ,1-KP2VLLC,1


In [62]:
df_S_CONTACT_QUOTE_CREATOR_TEAM.count()

QUOTECTRT_TEAM          840
QUOTECTRT_QUOTE_ID      840
QUOTECTRT_ORD_ROW_ID    840
QUOTECTRT_POSTN_RANK    840
dtype: int64

## S_SRC Campaign name from Order ORDCAMP

In [63]:
query_S_SRC_CAMPAIGN = ("SELECT ORD.ROW_ID ORDCAMP_ORD_ROW_ID, CAMP.NAME ORDCAMP_CAMPAIGN FROM S_SRC CAMP INNER JOIN S_ORDER ORD ON ORD.PROMO_ID = CAMP.ROW_ID WHERE %s" %(order_dt_filter))

df_S_SRC_CAMPAIGN = pd.read_sql(query_S_SRC_CAMPAIGN, cnxn)
df_S_SRC_CAMPAIGN.head(3)

,ORDCAMP_ORD_ROW_ID,ORDCAMP_CAMPAIGN


In [64]:
df_S_SRC_CAMPAIGN.count()

ORDCAMP_ORD_ROW_ID    0
ORDCAMP_CAMPAIGN      0
dtype: int64

## S_PTY_PAY_PREF Payment Preference from Order ORDPAYPREF

In [65]:
query_S_PTY_PAY_PREF_PAY_PREF = ("SELECT ORD.ROW_ID ORDPAYPREF_ORD_ROW_ID, PAYPRFL.PAY_METH_CD ORDPAYPREF_PAY_METH FROM S_PTY_PAY_PRFL PAYPRFL INNER JOIN S_ORDER ORD ON ORD.X_PAYMENT_PROFILE_ID = PAYPRFL.ROW_ID WHERE %s" %(order_dt_filter))

df_S_PTY_PAY_PREF_PAY_PREF = pd.read_sql(query_S_PTY_PAY_PREF_PAY_PREF, cnxn)
df_S_PTY_PAY_PREF_PAY_PREF.head(3)

,ORDPAYPREF_ORD_ROW_ID,ORDPAYPREF_PAY_METH
0,1-L27UV4O,Credit Card
1,1-L298WOI,Credit Card
2,1-L29944R,Credit Card


In [66]:
df_S_PTY_PAY_PREF_PAY_PREF.count()

ORDPAYPREF_ORD_ROW_ID    22252
ORDPAYPREF_PAY_METH      22252
dtype: int64

## REGGIE_COVER_SERIAL - Reggie serial numbers REGG

In [67]:
query_REGGIE_COVER_SERIALS = ("SELECT SERIAL_NUM REGG_SERIAL_NUM FROM REGGIE_COVER_SERIALS")

df_REGGIE_COVER_SERIALS = pd.read_sql(query_REGGIE_COVER_SERIALS, cnxn)
df_REGGIE_COVER_SERIALS.head(3)

,REGG_SERIAL_NUM
0,0000117
1,0121479
2,0146149


In [68]:
df_REGGIE_COVER_SERIALS.count()

REGG_SERIAL_NUM    270681
dtype: int64

## S_ORG_EXT - Order item owner account ORDITMOWNACC

In [69]:
query_S_ORG_EXT_ITEM_OWNER_ACCNT = ("SELECT ORD.ROW_ID ORDITMOWNACC_ORD_ROW_ID, ORDITEM.ROW_ID ORDITMOWNACC_ORDITEM_ROW_ID, ITEM_OWNER_ACC.INTEGRATION_ID ORDITMOWNACC_CLIENT_ID, ITEM_OWNER_ACC.NAME ORDITMOWNACC_NAME, ITEM_OWNER_ACC.X_DIVISION ORDITMOWNACC_DIVISION, ITEM_OWNER_ACC.X_SEGMENT ORDITMOWNACC_SEGMENT FROM S_ORG_EXT ITEM_OWNER_ACC INNER JOIN S_ORDER_ITEM ORDITEM ON ORDITEM.OWNER_ACCOUNT_ID = ITEM_OWNER_ACC.ROW_ID INNER JOIN S_ORDER ORD ON ORD.ROW_ID = ORDITEM.ORDER_ID INNER JOIN S_PROD_INT PROD ON ORDITEM.PROD_ID = PROD.ROW_ID AND PROD.TYPE IS NOT NULL INNER JOIN S_PROD_LN PRODLN ON PROD.PR_PROD_LN_ID = PRODLN.ROW_ID WHERE %s AND %s AND %s AND %s AND %s" %(order_dt_filter, order_status_filter, product_price_type_filter, orditem_prod_ln_filter,orditem_prod_part_num_filter))

df_S_ORG_EXT_ITEM_OWNER_ACCNT = pd.read_sql(query_S_ORG_EXT_ITEM_OWNER_ACCNT, cnxn)
df_S_ORG_EXT_ITEM_OWNER_ACCNT.head(3)

,ORDITMOWNACC_ORD_ROW_ID,ORDITMOWNACC_ORDITEM_ROW_ID,ORDITMOWNACC_CLIENT_ID,ORDITMOWNACC_NAME,ORDITMOWNACC_DIVISION,ORDITMOWNACC_SEGMENT
0,1-L4XRBQ0,1-L4XRBQK,8209631,ANZ,Business Division,Growing Business
1,1-L50E2IU,1-L50E2JE,8209631,ANZ,Business Division,Growing Business
2,1-L50R1UM,1-L50R1V6,8209631,ANZ,Business Division,Growing Business


In [70]:
df_S_ORG_EXT_ITEM_OWNER_ACCNT.count()

ORDITMOWNACC_ORD_ROW_ID        6821
ORDITMOWNACC_ORDITEM_ROW_ID    6821
ORDITMOWNACC_CLIENT_ID         6821
ORDITMOWNACC_NAME              6821
ORDITMOWNACC_DIVISION          6821
ORDITMOWNACC_SEGMENT           6821
dtype: int64

## S_POSTN - Order primary Position ORDPRPOS
1. Order position -> Order sales rep, team

In [71]:
query_S_POSTN_ORDER = ("SELECT ORD.ROW_ID ORDPRPOS_ORD_ROW_ID, ORDER_SALES_REP.FST_NAME + ' ' + ORDER_SALES_REP.LAST_NAME ORDPRPOS_SALES_REP, ORDER_TEAM.NAME ORDPRPOS_TEAM FROM S_ORDER ORD INNER JOIN S_POSTN ORDER_POS ON ORD.PR_POSTN_ID = ORDER_POS.ROW_ID left outer join S_CONTACT ORDER_SALES_REP ON ORDER_SALES_REP.ROW_ID = ORDER_POS.PR_EMP_ID left outer join S_ORG_EXT ORDER_TEAM ON ORDER_POS.OU_ID = ORDER_TEAM.ROW_ID WHERE %s" %(order_dt_filter))

df_S_POSTN_ORDER = pd.read_sql(query_S_POSTN_ORDER, cnxn)
df_S_POSTN_ORDER.head(3)

,ORDPRPOS_ORD_ROW_ID,ORDPRPOS_SALES_REP,ORDPRPOS_TEAM
0,1-KOZQVJI,Siebel Administrator,Siebel Administration
1,1-KPFM7LY,Siebel Administrator,Siebel Administration
2,1-KPMJRM0,Siebel Administrator,Siebel Administration


In [72]:
df_S_POSTN_ORDER.count()

ORDPRPOS_ORD_ROW_ID    23063
ORDPRPOS_SALES_REP     23063
ORDPRPOS_TEAM          23063
dtype: int64

## S_EVT_ACT - Order confirmation ORDCON
1. This is to state if the order has been confirmed by looking into Activities

In [73]:
query_S_EVT_ACT_ORDER = ("SELECT ORD.ROW_ID ORDCON_ORDER_ID, A.TARGET_OU_ID ORDCON_TARGET_OU_ID FROM S_EVT_ACT A INNER JOIN S_ORDER ORD ON ORD.BILL_ACCNT_ID = A.TARGET_OU_ID WHERE %s AND A.X_INTERACTION_CD = 'AD Partner Team' AND A.X_INTERACTION2_CD = 'Sub Order Confirm Received' GROUP BY ORD.ROW_ID, A.TARGET_OU_ID" %(order_dt_filter))

df_S_EVT_ACT_ORDER = pd.read_sql(query_S_EVT_ACT_ORDER, cnxn)
df_S_EVT_ACT_ORDER.head(3)

,ORDCON_ORDER_ID,ORDCON_TARGET_OU_ID


In [74]:
df_S_EVT_ACT_ORDER.count()

ORDCON_ORDER_ID        0
ORDCON_TARGET_OU_ID    0
dtype: int64

## S_CONTACT - Quote Refer Contact, Evanglist QUOTEREFEVA

In [75]:
query_S_CONTACT_QUOTE_REFER_CON = ("SELECT QUOTE.ROW_ID QUOTEREFEVA_QUOTE_ID, ORD.ROW_ID QUOTEREFEVA_ORD_ROW_ID, REF_CONTACT.FST_NAME + ' ' + REF_CONTACT.LAST_NAME QUOTEREFEVA_CONTACT FROM S_DOC_QUOTE QUOTE INNER JOIN S_CONTACT REF_CONTACT ON REF_CONTACT.ROW_ID = QUOTE.X_REF_CON_ID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID WHERE %s" %(order_dt_filter))

df_S_CONTACT_QUOTE_REFER_CON = pd.read_sql(query_S_CONTACT_QUOTE_REFER_CON, cnxn)
df_S_CONTACT_QUOTE_REFER_CON.head(3)

,QUOTEREFEVA_QUOTE_ID,QUOTEREFEVA_ORD_ROW_ID,QUOTEREFEVA_CONTACT
0,1-L27IBQB,1-L27ICAM,XNCGCKHWCU XNCGCKHWCU
1,1-L27IBQZ,1-L27ICB0,VEXXRVRSML VEXXRVRSML
2,1-LGZIJIG,1-LGZILOT,SUYFANDERY SUYFANDERY


In [76]:
df_S_CONTACT_QUOTE_REFER_CON.count()

QUOTEREFEVA_QUOTE_ID      53
QUOTEREFEVA_ORD_ROW_ID    53
QUOTEREFEVA_CONTACT       53
dtype: int64

## S_CONTACT - Quote Evanglist Contact QUOTEEVACON

In [77]:
query_S_CONTACT_QUOTE_EVAN_CON = ("SELECT QUOTE.ROW_ID QUOTEEVACON_QUOTE_ID, ORD.ROW_ID QUOTEEVACON_ORD_ROW_ID, QUOTE_EVANG_CON.FST_NAME + ' ' + QUOTE_EVANG_CON.LAST_NAME AS QUOTEEVACON_NAME, QUOTE_EVANG_CON_POSTN.NAME AS QUOTEEVACON_POSTN, QUOTE_EVANG_TEAM.NAME AS QUOTEEVACON_TEAM FROM S_DOC_QUOTE QUOTE INNER JOIN S_CONTACT QUOTE_EVANG_CON ON QUOTE_EVANG_CON.ROW_ID = QUOTE.X_PROD_EVNGLIST_ID left outer join S_POSTN QUOTE_EVANG_CON_POSTN ON QUOTE_EVANG_CON.PR_HELD_POSTN_ID = QUOTE_EVANG_CON_POSTN.ROW_ID LEFT OUTER JOIN S_PARTY QUOTE_EVANG_TEAM ON QUOTE_EVANG_CON_POSTN.OU_ID = QUOTE_EVANG_TEAM.PARTY_UID INNER JOIN S_ORDER ORD ON ORD.QUOTE_ID = QUOTE.ROW_ID WHERE %s" %(order_dt_filter))

df_S_CONTACT_QUOTE_EVAN_CON = pd.read_sql(query_S_CONTACT_QUOTE_EVAN_CON, cnxn)
df_S_CONTACT_QUOTE_EVAN_CON.head(3)

,QUOTEEVACON_QUOTE_ID,QUOTEEVACON_ORD_ROW_ID,QUOTEEVACON_NAME,QUOTEEVACON_POSTN,QUOTEEVACON_TEAM
0,1-L9QHNPC,1-L9QHOIQ,UATAD ADSCH7,AU AD Accountants Resourcing TM1,None
1,1-LALWY8Q,1-LALWYUW,UATAD ADSCH7,AU AD Accountants Resourcing TM1,None
2,1-LBHCCRE,1-LBHCH86,UATAD ADSCH7,AU AD Accountants Resourcing TM1,None


In [78]:
df_S_CONTACT_QUOTE_REFER_CON.count()

QUOTEREFEVA_QUOTE_ID      53
QUOTEREFEVA_ORD_ROW_ID    53
QUOTEREFEVA_CONTACT       53
dtype: int64

## ORDER REF ACCOUNT; AND EVANGELIST ORDREFACCEVA

In [79]:
query_S_ORG_EXT_ORDER_REF_ACCNT_EVAN = ("SELECT ORD.ROW_ID ORDREFACCEVA_ORD_ROW_ID,ORD.X_REF_ACCT_ID ORDREFACCEVA_REF_ACCT_ID,REF_EVANG.LOGIN AS ORDREFACCEVA_REF_EVANGELIST, REFACCOUNT_EVANG_CON.FST_NAME + ' ' + REFACCOUNT_EVANG_CON.LAST_NAME AS ORDREFACCEVA_CON_NAME,REFACCOUNT_EVANG_CON_POSTN.NAME AS ORDREFACCEVA_POSTN,REFACCOUNT_EVANG_TEAM.NAME AS ORDREFACCEVA_TEAM FROM S_ORDER ORD INNER JOIN S_ORG_EXT ORDREF ON ORD.X_REF_ACCT_ID = ORDREF.ROW_ID left outer join S_USER REF_EVANG ON ORDREF.X_PROD_EVNGLIST_ID = REF_EVANG.ROW_ID left outer join S_CONTACT REFACCOUNT_EVANG_CON ON ORDREF.X_PROD_EVNGLIST_ID = REFACCOUNT_EVANG_CON.ROW_ID left outer join S_POSTN REFACCOUNT_EVANG_CON_POSTN ON REFACCOUNT_EVANG_CON_POSTN.ROW_ID = REFACCOUNT_EVANG_CON.PR_HELD_POSTN_ID LEFT OUTER JOIN S_PARTY REFACCOUNT_EVANG_TEAM ON REFACCOUNT_EVANG_CON_POSTN.OU_ID = REFACCOUNT_EVANG_TEAM.PARTY_UID WHERE %s" %(order_dt_filter))

df_S_ORG_EXT_ORDER_REF_ACCNT_EVAN = pd.read_sql(query_S_ORG_EXT_ORDER_REF_ACCNT_EVAN, cnxn)
df_S_ORG_EXT_ORDER_REF_ACCNT_EVAN.head(3)

,ORDREFACCEVA_ORD_ROW_ID,ORDREFACCEVA_REF_ACCT_ID,ORDREFACCEVA_REF_EVANGELIST,ORDREFACCEVA_CON_NAME,ORDREFACCEVA_POSTN,ORDREFACCEVA_TEAM
0,1-KWQQHNB,1-6QMJ2QL,None,None,None,None
1,1-LHVQGX1,1-6QMJ2QL,None,None,None,None
2,1-L4YC5IL,1-906BRQM,None,None,None,None


In [80]:
df_S_ORG_EXT_ORDER_REF_ACCNT_EVAN.count()

ORDREFACCEVA_ORD_ROW_ID        154
ORDREFACCEVA_REF_ACCT_ID       154
ORDREFACCEVA_REF_EVANGELIST     82
ORDREFACCEVA_CON_NAME           82
ORDREFACCEVA_POSTN              82
ORDREFACCEVA_TEAM               48
dtype: int64

## AGREEMENT REF ACCOUNT ; AND EVANGELIST  AGRREFACC
1. Get the order header using the ORDER_ID in Agreement header table. Restrict the order by the order date.

#To filter not nulls
df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN = df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN.loc[(df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN['AGREEREF_CLIENT_ID'].notnull())]

#To filter nulls
df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN = df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN.loc[(df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN['AGREEREF_CLIENT_ID'].isnull())]

#NEED DISTINCT to stop duplicates when a single order creates 2 seperate agreements.

In [118]:
query_S_ORG_EXT_AGREE_REF_ACCNT_EVAN = ("SELECT DISTINCT ORD.ROW_ID AGRREFACC_ORD_ROW_ID,AGREEREF.INTEGRATION_ID AGRREFACC_CLIENT_ID ,AGREEREF.NAME AGRREFACC_NAME,AGREEREF.X_DIVISION AGRREFACC_DIVISION,AGREEREF.X_SEGMENT AGRREFACC_SEGMENT,AGREE_EVANG_CON.FST_NAME + ' ' + AGREE_EVANG_CON.LAST_NAME AGRREFACC_CON_NAME,AGREE_EVANG.LOGIN AGRREFACC_EVANGELIST,AGREE_EVANG_CON_POS.NAME AGRREFACC_POS,TEAM_AGREE_EVANG.NAME AGRREFACC_TEAM_NAME FROM S_ORDER ORD INNER JOIN S_DOC_AGREE AGR ON AGR.ORDER_ID = ORD.ROW_ID left outer join S_ORG_EXT AGREEREF ON AGR.X_REF_ACCT_ID = AGREEREF.ROW_ID left outer join S_USER AGREE_EVANG ON AGREEREF.X_PROD_EVNGLIST_ID = AGREE_EVANG.ROW_ID left outer join S_CONTACT AGREE_EVANG_CON ON AGREEREF.X_PROD_EVNGLIST_ID = AGREE_EVANG_CON.ROW_ID left outer join S_POSTN AGREE_EVANG_CON_POS ON AGREE_EVANG_CON.PR_HELD_POSTN_ID = AGREE_EVANG_CON_POS.ROW_ID LEFT OUTER JOIN S_PARTY TEAM_AGREE_EVANG ON AGREE_EVANG_CON_POS.OU_ID = TEAM_AGREE_EVANG.PARTY_UID WHERE %s" %(order_dt_filter))

df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN = pd.read_sql(query_S_ORG_EXT_AGREE_REF_ACCNT_EVAN, cnxn)
df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN.head(3)

,AGRREFACC_ORD_ROW_ID,AGRREFACC_CLIENT_ID,AGRREFACC_NAME,AGRREFACC_DIVISION,AGRREFACC_SEGMENT,AGRREFACC_CON_NAME,AGRREFACC_EVANGELIST,AGRREFACC_POS,AGRREFACC_TEAM_NAME
0,1-KP1SS9G,None,None,None,None,None,None,None,None
1,1-KP1W521,None,None,None,None,None,None,None,None
2,1-KP2M81D,None,None,None,None,None,None,None,None


In [119]:
df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN.count()

AGRREFACC_ORD_ROW_ID    3016
AGRREFACC_CLIENT_ID      107
AGRREFACC_NAME           107
AGRREFACC_DIVISION       107
AGRREFACC_SEGMENT        107
AGRREFACC_CON_NAME        47
AGRREFACC_EVANGELIST      47
AGRREFACC_POS             47
AGRREFACC_TEAM_NAME       47
dtype: int64

## Primary Agreement Flag PRIAGRFLG

In [83]:
query_S_DOC_AGREE_PRI_AGREE_FLG = ("SELECT ORD.ROW_ID PRIAGRFLG_ORD_ROW_ID, AGREE.ORDER_ID PRIAGRFLG_AGREE_ORDER_ID, AGREE_ITEM.X_ORDER_ITEM_ID PRIAGRFLG_ORDITM_ID, AGREE_ITEM.ROW_ID PRIAGRFLG_AGREE_ITEM_ID FROM S_AGREE_ITEM AGREE_ITEM LEFT OUTER JOIN S_DOC_AGREE AGREE ON AGREE.ROW_ID = AGREE_ITEM.DOC_AGREE_ID LEFT OUTER JOIN S_ORDER_ITEM ORDITEM ON ORDITEM.ROW_ID = AGREE_ITEM.X_ORDER_ITEM_ID INNER JOIN S_ORDER ORD ON ORDITEM.ORDER_ID = ORD.ROW_ID AND ORD.ROW_ID = AGREE.ORDER_ID WHERE %s" %(order_dt_filter))

df_S_DOC_AGREE_PRI_AGREE_FLG = pd.read_sql(query_S_DOC_AGREE_PRI_AGREE_FLG, cnxn)
df_S_DOC_AGREE_PRI_AGREE_FLG.head(3)

,PRIAGRFLG_ORD_ROW_ID,PRIAGRFLG_AGREE_ORDER_ID,PRIAGRFLG_ORDITM_ID,PRIAGRFLG_AGREE_ITEM_ID
0,1-KP1SS9G,1-KP1SS9G,1-KP1SS9J,1-KP1TOF1
1,1-KP1SS9G,1-KP1SS9G,1-KP1T104,1-KP1U29X
2,1-KP1W521,1-KP1W521,1-KP1W524,1-KP1WJW9


In [84]:
df_S_DOC_AGREE_PRI_AGREE_FLG.count()

PRIAGRFLG_ORD_ROW_ID        5073
PRIAGRFLG_AGREE_ORDER_ID    5073
PRIAGRFLG_ORDITM_ID         5073
PRIAGRFLG_AGREE_ITEM_ID     5073
dtype: int64

## joining the tables together
##295

In [124]:
df_SALES_MONITOR = pd.merge(df_S_ORDER_ITEM, df_S_ASSET, left_on = 'ORDITM_ROW_ID', right_on ='ORDITMASSET_ORDITEM_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORDER, left_on = 'ORDITM_ORDER_ID', right_on ='ORD_ORDER_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORDER_TYPE, left_on = 'ORDITM_ORDER_ID', right_on ='ORDTYPE_ORDER_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_PROD_INT, left_on = 'ORDITM_PROD_ID', right_on ='PROD_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_PROD_LN, left_on = 'PROD_LN_ID', right_on ='PRODLN_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORDER_ITEM_PAR_ITEM, left_on = 'ORDITM_ROW_ID', right_on ='PARORDITM_ORDITEM_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_INVOICE_ITEM, left_on = 'ORDITM_ROW_ID', right_on ='ORDITMINV_ORDITEM_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_INVOICE, left_on = 'ORDITM_ROW_ID', right_on ='INV_ORDITEM_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_PROD_INT_ORDITEM_PROMO, left_on = 'ORDITM_PROMOTION_ID', right_on ='ORDITMPPROD_PROD_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_PROMO_ITEM, left_on = 'ORDITMPPROD_PROD_ID', right_on ='PROMO_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORG_EXT_BILL, left_on = 'ORDITM_ORDER_ID', right_on ='BILLACC_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORG_EXT_ORD_ACCNT, left_on = 'ORDITM_ORDER_ID', right_on ='ORDACC_ORDER_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ASSET_PAR_ORD_ITEM_ASSET, left_on = 'ORDITM_ROW_ID', right_on ='PARORDITMAST_ORDITEM_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORG_EXT_LE, left_on = 'ORD_BU_ID', right_on ='LEGAL_ENTITY_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_AGREE_ITEM_ORD_INITIAL_AGREE, left_on = 'ORDITM_ROW_ID', right_on ='AGRITMEVG_ITEM_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_DOC_QUOTE, left_on = 'ORDITM_ORDER_ID', right_on ='QUOTE_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_POSTN_ACCT_MGR, left_on = 'ORDITM_ORDER_ID', right_on ='ACCMGR_ORDER_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_POSTN_QUOTE_SALES_REP, left_on = 'ORDITM_ORDER_ID', right_on ='QUOTEREP_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_SRC_CAMPAIGN, left_on = 'ORDITM_ORDER_ID', right_on ='ORDCAMP_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_PTY_PAY_PREF_PAY_PREF, left_on = 'ORDITM_ORDER_ID', right_on ='ORDPAYPREF_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_USER_QUOTE_EVANG, left_on = 'ORDITM_ORDER_ID', right_on ='QUOTEEVN_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_REGGIE_COVER_SERIALS, left_on = 'PARORDITMAST_ROW_ID', right_on ='REGG_SERIAL_NUM', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_CONTACT_Quote_Creater, left_on = 'ORDITM_ORDER_ID', right_on ='QUOTECTR_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_POSTN_ORDER, left_on = 'ORDITM_ORDER_ID', right_on ='ORDPRPOS_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_EVT_ACT_ORDER, left_on = 'ORDITM_ORDER_ID', right_on ='ORDCON_ORDER_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_CONTACT_QUOTE_REFER_CON, left_on = 'ORDITM_ORDER_ID', right_on ='QUOTEREFEVA_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_CONTACT_QUOTE_EVAN_CON, left_on = 'ORDITM_ORDER_ID', right_on ='QUOTEEVACON_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORG_EXT_ORDER_REF_ACCNT_EVAN, left_on = 'ORDITM_ORDER_ID', right_on ='ORDREFACCEVA_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_ORG_EXT_AGREE_REF_ACCNT_EVAN, left_on = 'ORDITM_ORDER_ID', right_on ='AGRREFACC_ORD_ROW_ID', how = 'left')
df_SALES_MONITOR = pd.merge(df_SALES_MONITOR, df_S_DOC_AGREE_PRI_AGREE_FLG, left_on = ['ORDITM_ORDER_ID', 'ORDITM_ROW_ID'], right_on =['PRIAGRFLG_ORD_ROW_ID','PRIAGRFLG_ORDITM_ID'], how = 'left')
df_SALES_MONITOR.head(3)

,ORDITM_ROW_ID,ORDITM_ADJ_UNIT_PRI,ORDITM_NET_PRI,ORDITM_ASSET_ID,ORDITM_ORDER_ID,ORDITM_PAR_ORDER_ITEM_ID,ORDITM_PROD_ID,ORDITM_PROMOTION_ID,ORDITM_OWNER_ACCOUNT_ID,ORDITM_QTY_REQ,...,AGRREFACC_DIVISION,AGRREFACC_SEGMENT,AGRREFACC_CON_NAME,AGRREFACC_EVANGELIST,AGRREFACC_POS,AGRREFACC_TEAM_NAME,PRIAGRFLG_ORD_ROW_ID,PRIAGRFLG_AGREE_ORDER_ID,PRIAGRFLG_ORDITM_ID,PRIAGRFLG_AGREE_ITEM_ID
0,1-KP1ZHAD,120.000,120.000,None,1-KP1ZHAB,None,1-KP1WPJZ,None,1-KP1YMMI,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1-KP31TLS,0.000,0.000,1-KP31BFT,1-KP31TLQ,None,1-11XL7K5,None,1-KP30OAA,1.0,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,1-KP31TLT,90.909,90.909,None,1-KP31TLQ,1-KP31TLS,1-11XL76S,None,1-KP30OAA,1.0,...,None,None,None,None,None,None,1-KP31TLQ,1-KP31TLQ,1-KP31TLT,1-KP329ST


In [125]:
df_SALES_MONITOR.count()

ORDITM_ROW_ID                  6820
ORDITM_ADJ_UNIT_PRI            5746
ORDITM_NET_PRI                 5804
ORDITM_ASSET_ID                2140
ORDITM_ORDER_ID                6820
ORDITM_PAR_ORDER_ITEM_ID       4407
ORDITM_PROD_ID                 6820
ORDITM_PROMOTION_ID            1236
ORDITM_OWNER_ACCOUNT_ID        6820
ORDITM_QTY_REQ                 6820
ORDITMASSET_ASSET_ROW_ID       2140
ORDITMASSET_REG_DATE           2140
ORDITMASSET_ORDITEM_ID         2140
ORD_ORDER_ID                   6820
ORD_BU_ID                      6820
ORD_ORDER_TYPE_ID              6820
ORD_BILL_ACCNT_ID              6820
ORD_ACCNT_ID                   6820
ORD_QUOTE_ID                   4513
ORD_PROMO_ID                      0
ORD_X_PAYMENT_PROFILE_ID       6566
ORD_PR_POSTN_ID                6820
ORD_ORDER_DT                   6820
ORD_STATUS_CD                  6820
ORDTYPE_ID                     6756
ORDTYPE_NAME                   6756
ORDTYPE_ORDER_ID               6756
PROD_ID                     